# The AURA Brain experiment #7
### 2025-11-16
### Author: Nicolas Cloutier
### Entity: Cognitiv Aura
### License: Apache 2.0
*** note: not for production use, for experimentation only contact the owner for production use ***


In [ ]:
import unsloth
# --- uninstall conflicting wandb ---
#!pip uninstall -y wandb

# --- 1. Install All Dependencies ---
!pip install "unsloth[colab-new]"
!pip install transformers torch sentence-transformers accelerate huggingface_hub datasets scipy scikit-learn -q

# --- 2. Import Core Libraries (for the whole notebook) ---
import uuid
import enum
from enum import Enum # Explicitly import Enum as requested
import numpy as np
import random
import json
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, logging
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import os
from unsloth import FastLanguageModel
from typing import List, Dict, Optional, Union, Any, Tuple, Literal
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from numpy import ndarray
import asyncio
import io
import csv
import threading # For Memory Pool
import time
from scipy.signal import hilbert # For Temporal Interpolator
from scipy.linalg import expm # For Temporal Interpolator
import glob # For finding latest keyframe
from datasets import load_dataset # For GoEmotions
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import nest_asyncio

# Suppress transformer warnings
logging.set_verbosity_error()

# --- 3. Colab/HF Login ---
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("Hugging Face login successful!")
except (ImportError, KeyError):
    print("HF_TOKEN secret not found or not in Colab. Ensure you are logged in.")

# --- 4. Mount Google Drive ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_DIR = "/content/drive/MyDrive/aura_education_v7_final" # New save dir
    print(f"Google Drive mounted. Will save/load brain from: {SAVE_DIR}")
except ImportError:
    print("Not in Colab. Saving to local directory './aura_education_v7_final'")
    SAVE_DIR = "./aura_education_v7_final"

os.makedirs(SAVE_DIR, exist_ok=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Hugging Face login successful!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted. Will save/load brain from: /content/drive/MyDrive/aura_education_v7_final


# Core Helpers and SNN Components

In [ ]:
# --- All Imports ---
import uuid
import enum
from enum import Enum # Explicit import
import numpy as np
import random
import json
import itertools
import torch
from transformers import AutoTokenizer, logging
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import os
from unsloth import FastLanguageModel
from typing import List, Dict, Optional, Union, Any, Tuple, Literal # Explicit imports
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from numpy import ndarray
import asyncio
import io
import csv
import threading # For Memory Pool
import time
import re
from collections import defaultdict, deque
from scipy.signal import hilbert # For Temporal Interpolator
from scipy.linalg import expm # For Temporal Interpolator
import glob # For finding latest keyframe
from datasets import load_dataset # For GoEmotions

# --- PyTorch Imports (for Pre-Training) ---
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# Suppress transformer warnings
logging.set_verbosity_error()

# --- 0. Colab/HF Login ---
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("Hugging Face login successful!")
except (ImportError, KeyError):
    print("HF_TOKEN secret not found or not in Colab. Ensure you are logged in.")

# --- 1. Helper Functions (Softplus, Tanh, Softmax) ---
def softplus(z: np.ndarray) -> np.ndarray:
    return np.log1p(np.exp(-np.abs(z))) + np.maximum(z, 0)
def tanh(z: np.ndarray) -> np.ndarray:
    return np.tanh(z)
def softmax(z: np.ndarray, temp: float = 1.0) -> np.ndarray:
    z = z / max(1e-8, temp); z = z - np.max(z); ez = np.exp(z)
    s = ez.sum(); return ez / (s + 1e-12)

def maybe_empty_cuda_cache(reason: str = "", min_free_ratio: float = 0.12) -> None:
    """Only clear CUDA cache if free VRAM drops below a ratio to avoid thrashing."""
    if not torch.cuda.is_available():
        return
    try:
        free_bytes, total_bytes = torch.cuda.mem_get_info()
        free_ratio = free_bytes / float(total_bytes)
        if free_ratio < min_free_ratio:
            tag = f" ({reason})" if reason else ""
            print(f"🔄 Clearing CUDA cache{tag}. Free VRAM ratio: {free_ratio:.3f}")
            torch.cuda.empty_cache()
    except Exception as exc:
        print(f"⚠️ Unable to query CUDA memory for cleanup: {exc}")

# --- 2. EnergyMeter ---
@dataclass
class EnergyMeter:
    e_mac_j: float = 3e-12; total_j: float = 0.0
    def add_macs(self, nmacs: int): self.total_j += self.e_mac_j * float(nmacs)

# --- 3. Memory Pool (from memory_pool.py) ---
@dataclass
class PoolStats:
    hits: int = 0; misses: int = 0; total_allocations: int = 0; peak_usage_mb: float = 0.0
class ArrayPool:
    """"""
    def __init__(self, max_pool_size_mb: int = 512):
        self.max_pool_size = max_pool_size_mb * 1024 * 1024
        self.pools: Dict[Tuple[tuple, np.dtype], List[np.ndarray]] = {}
        self.current_usage = 0; self.stats = PoolStats(); self._lock = threading.Lock()
    def get_array(self, shape: tuple, dtype: np.dtype = np.float32, zero_fill: bool = True) -> np.ndarray:
        key = (shape, dtype)
        with self._lock:
            if key in self.pools and self.pools[key]:
                arr = self.pools[key].pop(); self.stats.hits += 1
                if zero_fill: arr.fill(0); return arr
            else:
                arr = np.empty(shape, dtype=dtype);
                if zero_fill: arr.fill(0)
                self.stats.misses += 1; self.stats.total_allocations += 1; return arr
    def return_array(self, arr: np.ndarray) -> None:
        if arr is None: return
        key = (arr.shape, arr.dtype); array_size = arr.nbytes
        with self._lock:
            if self.current_usage + array_size <= self.max_pool_size:
                if key not in self.pools: self.pools[key] = []
                arr.fill(0); self.pools[key].append(arr); self.current_usage += array_size
_global_pool = ArrayPool()
def get_pooled_array(shape: tuple, dtype: np.dtype = np.float32, zero_fill: bool = True) -> np.ndarray:
    return _global_pool.get_array(shape, dtype, zero_fill)
def return_pooled_array(arr: np.ndarray) -> None:
    _global_pool.return_array(arr)

# --- 4. Optimized Whitener (from training_coordinator_optimized.py) ---
class OptimizedWhitener:
    """"""
    def __init__(self, dim: int, eps: float = 1e-6, momentum: float = 0.01):
        self.dim = dim; self.eps = np.float32(eps); self.momentum = np.float32(momentum)
        self.mu = np.zeros(dim, dtype=np.float32); self.var = np.ones(dim, dtype=np.float32)
        self._temp_diff = np.zeros(dim, dtype=np.float32)
        self._temp_result = np.zeros(dim, dtype=np.float32)
    def transform(self, x: np.ndarray) -> np.ndarray:
        if x.dtype != np.float32: x = x.astype(np.float32)
        self.mu *= (1.0 - self.momentum); self.mu += self.momentum * x
        np.subtract(x, self.mu, out=self._temp_diff)
        np.multiply(self._temp_diff, self._temp_diff, out=self._temp_result)
        self.var *= (1.0 - self.momentum); self.var += self.momentum * self._temp_result
        np.sqrt(self.var + self.eps, out=self._temp_result)
        np.divide(self._temp_diff, self._temp_result, out=self._temp_result)
        return self._temp_result.copy()
    def state_dict(self) -> Dict: return {"mu": self.mu, "var": self.var}
    def load_state_dict(self, state: Dict): self.mu = state["mu"]; self.var = state["var"]

# --- 5. GoEmotions Labels (The Curriculum) ---
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
GOEMOTIONS_MAP = {label: idx for idx, label in enumerate(GOEMOTIONS_LABELS)}
NUM_GOEMOTIONS = len(GOEMOTIONS_LABELS)

def quantize_moe_prediction(prediction: float, num_labels: int) -> Tuple[int, float]:
    """Round/clamp a continuous MoE prediction to a valid label index."""
    if np.isnan(prediction) or np.isinf(prediction):
        prediction = 0.0
    discrete_idx = int(np.clip(round(prediction), 0, num_labels - 1))
    return discrete_idx, float(discrete_idx)

# --- 6. "Ears": FeatureGenerator (GoEmotions-Aware) ---
class FeatureGenerator:
    """Based on clean_amygdala_trainer.py"""
    def __init__(self, sbert_model_name: str = "all-MiniLM-L6-v2"):
        self.SBERT_DIM = 384; self.SINE_LENGTH = 32; self.EXTRA_FEATURES = 3
        self.TOTAL_FEATURES = self.SBERT_DIM + self.SINE_LENGTH + self.EXTRA_FEATURES # 419
        print(f"CREATED: FeatureGenerator (Aura 7.0 Ears), Features: {self.TOTAL_FEATURES}")
        self.sbert_model = SentenceTransformer(sbert_model_name, device="cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = self.sbert_model.tokenizer
        self.vocab_size = self.tokenizer.vocab_size
        self.label_params = self._generate_default_params(GOEMOTIONS_LABELS)
        self.whitener = OptimizedWhitener(dim=self.TOTAL_FEATURES)

    def _generate_default_params(self, labels: List[str]) -> Dict[str, Dict]:
        params = {}
        for idx, label in enumerate(labels):
            params[label] = {"freq": 1.5 + 0.1 * idx, "amp": 0.7, "phase": 0.5 + 0.2 * idx}
        return params

    def build_features(self, record: Dict[str, Any], sbert_vec: np.ndarray, use_emotion_sine: bool = True) -> np.ndarray:
        """
        Builds the feature vector.
        If use_emotion_sine is False, it forces a neutral sine wave.
        This is the FIX to prevent data leakage to the Thalamus.
        """
        if use_emotion_sine:
            prim = record.get("plutchik", {}).get("primary", "neutral")
        else:
            prim = "neutral" # <-- THE FIX

        cfg = self.label_params.get(prim, {"freq": 1.0, "amp": 0.0, "phase": 0.0})
        t = np.linspace(0, 2*np.pi, self.SINE_LENGTH, dtype=np.float32)
        emb = (cfg["amp"] * np.sin(cfg["freq"] * t + cfg["phase"])).astype(np.float32)
        text = record.get("text", "")
        extras = np.array([len(text) / 100.0, int("!" in text), int("?" in text)], dtype=np.float32)
        return np.concatenate([emb, extras, sbert_vec]).astype(np.float32)

    def generate_for_query(self, query: str) -> (np.ndarray, List[int]):
        """
        Generates the feature vector for the *real-time* brain (Hebbian/MoE).
        This version *can* use an emotional sine wave, as it's part of the context.
        """
        record = {"text": query, "plutchik": {"primary": "neutral"}} # Live query is "neutral" by default
        sbert_vec = self.sbert_model.encode(query, normalize_embeddings=True)
        # We use use_emotion_sine=True (default)
        x_raw = self.build_features(record, sbert_vec, use_emotion_sine=True)
        x_whitened = self.whitener.transform(x_raw)
        token_ids = self.tokenizer.encode(query, add_special_tokens=False)
        return x_whitened, token_ids

    def state_dict(self) -> Dict: return {"whitener_mu": self.whitener.mu, "whitener_var": self.whitener.var}
    def load_state_dict(self, state: Dict): self.whitener.mu = state["mu"]; self.whitener.var = state["var"]

# --- 7. "Mouth": ResponseGenerator (Unsloth, Async) ---
class ResponseGenerator:
    def __init__(self, model_name: str = "meta-llama/Llama-3.2-3B-Instruct"):
        print(f"CREATED: ResponseGenerator (Mouth) using Unsloth on '{model_name}'")
        max_seq_length = 2048; dtype = None; load_in_4bit = True
        unsloth_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
        print(f"   -> Using optimized Unsloth 4-bit model: {unsloth_model_name}")
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name = unsloth_model_name, max_seq_length = max_seq_length,
            dtype = dtype, load_in_4bit = load_in_4bit,
        )
        if self.tokenizer.pad_token is None: self.tokenizer.pad_token = self.tokenizer.eos_token
        print("   -> Unsloth Llama 3.2 model loaded successfully.")
    def _build_prompt(self, user_query: str, brain_state: dict) -> str:
        cns_state = brain_state.get('cns_state', 'ALERT').name
        stress = brain_state.get('stress_level', 0.0)
        persona = f"You are Aura, a bio-neural AI. Your current internal state is {cns_state}."
        if cns_state == 'HYPERVIGILANT' or stress > 1.0:
            persona += f" You are feeling a high-stress level ({stress:.2f})."
        else: persona += " You are calm and helpful."
        messages = [{"role": "system", "content": persona}, {"role": "user", "content": user_query}]
        return self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    async def generate_response(self, user_query: str, brain_state: dict) -> str:
        print(f"\n--- 👄 ResponseGenerator (Unsloth) ---"); print(f"Generating response. Brain state: {brain_state}")
        prompt = self._build_prompt(user_query, brain_state)
        inputs = self.tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")
        terminators = [self.tokenizer.eos_token_id, self.tokenizer.convert_tokens_to_ids("<|eot_id|>")]
        outputs = await asyncio.to_thread(
            self.model.generate, **inputs, max_new_tokens=150, eos_token_id=terminators,
            do_sample=True, temperature=0.7, top_p=0.9,
        )
        response_text = self.tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[-1]:], skip_special_tokens=True)[0]
        print(f"Llama 3.2 (Unsloth) Output: {response_text}"); return response_text

# --- 8. SpikingAttention (k-WTA) ---
@dataclass
class SpikingAttention:
    decay: float = 0.7; theta: float = 1.0; k_winners: int = 5
    gain_up: float = 1.5; gain_down: float = 0.6
    def compute_gains(self, token_seq: List[int], vocab_size: int) -> Optional[np.ndarray]:
        if not token_seq: return None
        v: Dict[int, float] = {}; spikes: Dict[int, int] = {}
        for j in token_seq:
            vj = self.decay * v.get(j, 0.0) + 1.0
            if vj >= self.theta: spikes[j] = spikes.get(j, 0) + 1; vj -= self.theta
            v[j] = vj
        ranked = sorted(spikes.items(), key=lambda kv: (-kv[1], -v.get(kv[0], 0.0)))
        winners = set(j for j,_ in ranked[:max(1, self.k_winners)])
        gains = np.ones(vocab_size, dtype=np.float64)
        seen = set(spikes.keys()) | set(v.keys())
        for j in seen:
            if 0 <= j < vocab_size: gains[j] = self.gain_up if j in winners else self.gain_down
        return gains

@dataclass
class MultiChannelSpikingAttention:
    """Fuse amplitude/pitch/boundary spike trains into k-WTA attention gains."""
    k_winners: int = 5
    decay_amp: float = 0.7; theta_amp: float = 1.0
    decay_pitch: float = 0.7; theta_pitch: float = 1.0
    decay_bound: float = 0.7; theta_bound: float = 1.0
    w_amp: float = 1.0; w_pitch: float = 1.0; w_bound: float = 1.0
    gain_up: float = 1.8; gain_down: float = 0.6
    min_gain: float = 0.2; max_gain: float = 3.0
    smoothing: int = 0
    normalize_salience: bool = True
    use_stdp: bool = False
    lr_plus: float = 0.05; lr_minus: float = 0.03
    w_min: float = -1.0; w_max: float = 1.0
    stdp_alpha: float = 0.2
    token_weights: Optional[Dict[int, float]] = None

    def _lif(self, x: np.ndarray, decay: float, theta: float) -> np.ndarray:
        v = 0.0; out = np.zeros_like(x, dtype=np.float64)
        for i, xi in enumerate(np.asarray(x, dtype=np.float64)):
            v = decay * v + xi
            if v >= theta:
                out[i] = 1.0
                v -= theta
        return out

    def _compute_spikes(self, seq: np.ndarray, decay: float, theta: float) -> np.ndarray:
        return self._lif(seq, decay, theta)

    def compute(
        self,
        token_ids: List[int],
        amp: Optional[np.ndarray],
        pitch: Optional[np.ndarray],
        boundary: Optional[np.ndarray],
    ) -> Optional[Dict[str, Any]]:
        if amp is None or pitch is None or boundary is None:
            return None
        if not token_ids:
            return None
        T = len(token_ids)
        if amp.shape[0] != T or pitch.shape[0] != T or boundary.shape[0] != T:
            return None

        s_amp = self._compute_spikes(amp, self.decay_amp, self.theta_amp)
        s_pitch = self._compute_spikes(pitch, self.decay_pitch, self.theta_pitch)
        s_bound = self._compute_spikes(boundary, self.decay_bound, self.theta_bound)
        sal = self.w_amp * s_amp + self.w_pitch * s_pitch + self.w_bound * s_bound

        stdp_mod = None
        if self.use_stdp:
            if self.token_weights is None:
                self.token_weights = {}
            spiked = (s_amp + s_pitch + s_bound) > 0
            for i, tok in enumerate(token_ids):
                prev = self.token_weights.get(tok, 0.0)
                delta = self.lr_plus if spiked[i] else -self.lr_minus
                new_w = np.clip(prev + delta, self.w_min, self.w_max)
                self.token_weights[tok] = float(new_w)
            stdp_mod = np.ones_like(sal, dtype=np.float64)
            for i, tok in enumerate(token_ids):
                w_tok = self.token_weights.get(tok, 0.0)
                stdp_mod[i] = max(0.0, 1.0 + self.stdp_alpha * w_tok)

        if self.smoothing and self.smoothing > 1:
            k = int(self.smoothing)
            kernel = np.ones(k, dtype=np.float64) / k
            sal = np.convolve(sal, kernel, mode="same")

        if self.normalize_salience and len(sal) > 0:
            m = float(np.max(sal))
            if m > 0: sal = sal / m

        if stdp_mod is not None:
            sal = sal * stdp_mod

        order = np.argsort(-sal)
        winners_idx = [int(i) for i in order[:max(1, self.k_winners)] if sal[int(i)] > 0]
        seen_idx = np.nonzero(sal > 0)[0].tolist()

        mu_scalar = 1.0
        if winners_idx:
            mu_scalar = float(np.clip(np.mean(sal[winners_idx]) * self.gain_up, self.min_gain, self.max_gain))

        if not winners_idx and seen_idx:
            loser_sal = np.mean(sal[seen_idx])
            mu_scalar = float(np.clip(loser_sal * self.gain_down, self.min_gain, self.max_gain))

        return {
            "mu_scalar": mu_scalar,
            "salience": sal,
            "winners_idx": winners_idx,
            "seen_idx": seen_idx,
        }

def prosody_channels_from_text(tokens: List[str]) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Extract crude amplitude/pitch/boundary channels from word tokens."""
    T = len(tokens)
    amp = np.zeros(T, dtype=np.float64)
    pitch = np.zeros(T, dtype=np.float64)
    boundary = np.zeros(T, dtype=np.float64)
    emotive = {
        "wow","amazing","terrible","great","awful","love","hate","yay","ugh",
        "fantastic","horrible","incredible","disgusting","beautiful","ugly"
    }
    for i, tok in enumerate(tokens):
        base = tok.strip()
        amp[i] = 1.0 if (base.isupper() and len(base) > 2) or ("!" in base) else 0.0
        pitch[i] = 1.0 if ("?" in base or base.lower() in emotive or any(ch in base for ch in "😊😂😭😡🎉🔥💯")) else 0.0
        boundary[i] = 1.0 if any(ch in base for ch in ".!?;,:") else 0.0
    return amp, pitch, boundary

def simple_tokenize_for_attention(text: str) -> List[str]:
    if not text:
        return []
    tokens = re.findall(r"\w+|[^\w\s]", text)
    return tokens if tokens else [text]

# --- 9. Hebbian Layer (Oja) ---
@dataclass
class OjaStepOut:
    y: np.ndarray; residual_ema: float; grew: bool
class OjaLayer:
    def __init__(self, dim: int, n_components: int = 8, lr: float = 5e-4, mode: str = "nonlinear",
                 *, max_components: int = 64, lateral_beta: float = 0.05,
                 grow_threshold: float = 0.35, ema: float = 0.01, grow_cooldown: int = 100,
                 seed: Optional[int] = 1337):
        self.rng = np.random.default_rng(seed)
        self.dim = int(dim); self.lr = float(lr); self.mode = str(mode)
        self.max_components = int(max_components); self.beta = float(lateral_beta)
        self.grow_threshold = float(grow_threshold); self.ema = float(ema)
        self.grow_cooldown = int(grow_cooldown); self.cooldown = 0
        W0 = self.rng.normal(0, 0.1, (n_components, self.dim))
        self.W = (W0.T / (np.linalg.norm(W0, axis=1) + 1e-12)).T
        self.K = self.W.shape[0]; self.residual_ema = 0.0; self._steps = 0
        print(f"CREATED: OjaLayer (Hebbian Cortex) with {self.K} components, mode='{self.mode}'")
    def step(self, xw: np.ndarray) -> OjaStepOut:
        x = np.asarray(xw, dtype=np.float64); y = self.W @ x; x_hat = self.W.T @ y
        xn = float(np.dot(x, x) + 1e-12); explained = float(np.dot(x_hat, x_hat) / xn)
        residual = float(1.0 - explained)
        if self.mode == "nonlinear":
            g = y ** 3; dW = (g[:, None] * x[None, :]) - self.W
        else: # sanger
            proj = np.zeros_like(x); dW = np.zeros_like(self.W)
            for i in range(self.K):
                proj = proj + y[i] * self.W[i]; dW[i] = y[i] * (x - proj)
        YW = y @ self.W; cross = YW[None, :] - (y[:, None] * self.W)
        dW -= self.beta * (y[:, None] * cross); self.W += self.lr * dW; self._renorm_rows()
        self.residual_ema = (1.0 - self.ema) * self.residual_ema + self.ema * residual
        grew, _ = self._maybe_grow(x, x_hat); self._steps += 1
        return OjaStepOut(y=y, residual_ema=self.residual_ema, grew=grew)
    def _renorm_rows(self): self.W = (self.W.T / (np.linalg.norm(self.W, axis=1) + 1e-12)).T
    def _maybe_grow(self, x: np.ndarray, x_hat: np.ndarray) -> Tuple[bool, Optional[int]]:
        if self.cooldown > 0: self.cooldown -= 1; return False, None
        if self.K >= self.max_components: return False, None
        if self.residual_ema < self.grow_threshold: return False, None
        r = x - x_hat; nr = float(np.linalg.norm(r))
        if nr > 1e-9: w_new = r / nr
        else:
            w_new = self.rng.normal(0, 0.1, size=self.dim)
            w_new /= (np.linalg.norm(w_new) + 1e-12)
        self.W = np.vstack([self.W, w_new]); self.K += 1
        self.cooldown = self.grow_cooldown
        print(f"🧬 OJA NEUROGENESIS: Residual high. Growing new component. K={self.K}"); return True, self.K - 1
    def state_dict(self) -> Dict: return {"W": self.W, "residual_ema": self.residual_ema, "_steps": self._steps}
    def load_state_dict(self, state: Dict):
        self.W = state["W"]; self.K = state["W"].shape[0]; self.residual_ema = state["residual_ema"]; self._steps = state["_steps"]

# --- 10. "Expert Neuron": ExpertNLMSHead ---
class ExpertNLMSHead:
    def __init__(self, n_features: int, vocab_size: int,
                 attention_config: Dict, mu: float = 0.1, epsilon: float = 1e-6,
                 mu_decay: float = 0.9995, mu_min: float = 0.01, initial_bias: float = 0.0,
                 guard_sign: bool = False, error_thresh: float = 0.08,
                 arousal_band: Optional[tuple] = None):
        self.n_features = n_features
        self.w = np.zeros(self.n_features, dtype=np.float64)
        self.mu_initial = mu
        self.mu = mu
        self.mu_decay = mu_decay
        self.mu_min = mu_min
        self.epsilon = epsilon
        self.vocab_size = vocab_size
        spiking_cfg = {k: v for k, v in attention_config.items() if k in ('decay', 'theta', 'k_winners', 'gain_up', 'gain_down')}
        if not spiking_cfg:
            spiking_cfg = {'decay': 0.7, 'theta': 1.0, 'k_winners': 5, 'gain_up': 1.5, 'gain_down': 0.6}
        self.spiking_attention = SpikingAttention(**spiking_cfg)
        multi_cfg = attention_config.get('multi_channel') if isinstance(attention_config, dict) else None
        self.multi_attention = MultiChannelSpikingAttention(**multi_cfg) if multi_cfg else None
        self.last_error = 0.0
        self.last_output = 0.0
        self.update_count = 0
        self.total_error_sq = 0.0
        self._lock = asyncio.Lock()
        self.bias = initial_bias  # Add bias term for better learning
        # Guard conditions from nlms.py
        self.guard_sign = guard_sign  # Skip update if sign mismatch but error small
        self.error_thresh = error_thresh  # Threshold for guard conditions
        self.arousal_band = arousal_band  # (lo, hi) band for arousal-based skipping
    def predict(self, x: np.ndarray) -> float:
        return float(np.dot(self.w, x) + self.bias)
    async def update(self, x: np.ndarray, y_true: float, token_ids: List[int],
                     attention_bundle: Optional[Dict[str, Any]] = None) -> float:
        async with self._lock:
            y_hat = self.predict(x)
            self.last_output = y_hat
            error = y_true - y_hat
            self.last_error = error
            self.total_error_sq += error * error
            self.update_count += 1

            # Guard conditions from nlms.py - skip update if conditions met
            if self.guard_sign:
                # Skip if sign mismatch but error is small (prevents unnecessary updates)
                if (y_hat > 0) != (y_true > 0) and abs(error) < self.error_thresh:
                    return y_hat

            if self.arousal_band is not None:
                # Skip if target is in arousal band and error is small
                lo, hi = self.arousal_band
                if (lo <= y_true <= hi) and abs(error) < self.error_thresh:
                    return y_hat

            attention_gains = self.spiking_attention.compute_gains(token_ids, self.vocab_size)
            avg_gain = 1.0
            if attention_gains is not None and token_ids:
                gains_for_seq = [attention_gains[token] for token in token_ids if 0 <= token < self.vocab_size]
                if gains_for_seq: avg_gain = np.mean(gains_for_seq)

            multi_mu_scalar = 1.0
            if attention_bundle is not None and self.multi_attention is not None:
                bundle_tokens = attention_bundle.get('token_ids', token_ids)
                mc_result = self.multi_attention.compute(
                    bundle_tokens,
                    attention_bundle.get('amp'),
                    attention_bundle.get('pitch'),
                    attention_bundle.get('boundary'),
                )
                if mc_result:
                    multi_mu_scalar = mc_result.get('mu_scalar', 1.0)

            self.mu = max(self.mu_min, self.mu * self.mu_decay)
            modulated_mu = self.mu * avg_gain * multi_mu_scalar

            # Adaptive learning rate based on error magnitude (larger errors = faster learning)
            error_magnitude = abs(error)
            adaptive_factor = 1.0 + min(2.0, error_magnitude / 10.0)  # Boost LR for large errors
            modulated_mu *= adaptive_factor

            # Clip error to prevent extreme updates (but allow larger errors)
            clipped_error = np.clip(error, -100.0, 100.0)

            x_norm_sq = np.dot(x, x) + self.epsilon
            step = (modulated_mu * clipped_error * x) / x_norm_sq

            # Clip step size to prevent divergence (increased threshold)
            step_norm = np.linalg.norm(step)
            max_step_norm = 3.0  # Increased further to allow larger updates
            if step_norm > max_step_norm:
                step = step * (max_step_norm / step_norm)

            self.w += step

            # Update bias term with adaptive learning (more aggressive for large errors)
            bias_learning_rate = 0.3 * adaptive_factor  # More aggressive bias learning
            bias_step = modulated_mu * clipped_error * bias_learning_rate
            self.bias += bias_step

            # Clip weights to prevent unbounded growth
            weight_norm = np.linalg.norm(self.w)
            max_weight_norm = 100.0
            if weight_norm > max_weight_norm:
                self.w = self.w * (max_weight_norm / weight_norm)

            # Clip bias to reasonable range
            self.bias = np.clip(self.bias, -50.0, 50.0)

            return y_hat
    def get_rmse(self) -> float:
        if self.update_count == 0: return float('inf')
        return float(np.sqrt(self.total_error_sq / self.update_count))
    def state_dict(self) -> Dict:
        return {"w": self.w, "bias": self.bias, "mu": self.mu, "update_count": self.update_count}
    def load_state_dict(self, state: Dict):
        self.w = state["w"]
        self.bias = state.get("bias", 0.0)
        self.mu = state.get("mu", self.mu_initial)
        self.update_count = state.get("update_count", 0)

# --- 11. Liquid/MoE Components ---
# Bandit-based gating for expert selection
class BanditGating:
    """UCB-based bandit gating to track and select best-performing experts"""
    def __init__(self, n_experts: int, exploration_factor: float = 2.0):
        self.n_experts = n_experts
        self.exploration_factor = exploration_factor
        # Track rewards (negative errors) and counts for each expert
        self.total_rewards = np.zeros(n_experts, dtype=np.float64)
        self.selection_counts = np.ones(n_experts, dtype=np.float64)  # Start at 1 to avoid division by zero
        self.total_selections = n_experts  # Total times any expert was selected

    def update(self, expert_idx: int, error: float):
        """Update bandit statistics with expert performance"""
        # Convert error to reward (lower error = higher reward)
        reward = 1.0 / (1.0 + abs(error))  # Reward in [0, 1]
        self.total_rewards[expert_idx] += reward
        self.selection_counts[expert_idx] += 1.0
        self.total_selections += 1

    def get_ucb_scores(self) -> np.ndarray:
        """Compute UCB scores for all experts"""
        # Average reward
        avg_rewards = self.total_rewards / self.selection_counts

        # Confidence bound
        confidence = self.exploration_factor * np.sqrt(
            np.log(self.total_selections + 1) / self.selection_counts
        )

        # UCB = average reward + confidence bound
        ucb_scores = avg_rewards + confidence
        return ucb_scores

    def select_top_k(self, k: int, base_gates: np.ndarray) -> tuple:
        """Select top-k experts using UCB scores, modulated by base gates"""
        ucb_scores = self.get_ucb_scores()

        # Combine UCB scores with base gates (weighted combination)
        # UCB provides exploration/exploitation, gates provide input-specific routing
        combined_scores = 0.7 * base_gates + 0.3 * ucb_scores

        # Select top-k
        topk_idx = np.argsort(combined_scores)[-k:][::-1]

        # Normalize gates
        topk_gates = combined_scores[topk_idx]
        topk_gates = np.maximum(topk_gates, 0.0)  # Ensure non-negative
        gate_sum = np.sum(topk_gates)
        if gate_sum > 0:
            topk_gates = topk_gates / gate_sum
        else:
            topk_gates = np.ones(k) / k  # Uniform if all zero

        return topk_idx, topk_gates

    def reset(self):
        """Reset bandit statistics"""
        self.total_rewards.fill(0.0)
        self.selection_counts.fill(1.0)
        self.total_selections = self.n_experts

@dataclass
class LiquidCell:
    in_dim: int; hidden_dim: int; dt: float = 0.02
    tau_min: float = 0.02; tau_max: float = 2.0
    rng: np.random.Generator = field(default_factory=lambda: np.random.default_rng(1337))
    W: np.ndarray = field(init=False); U: np.ndarray = field(init=False)
    b: np.ndarray = field(init=False); V: np.ndarray = field(init=False)
    c: np.ndarray = field(init=False); h: np.ndarray = field(init=False)
    def __post_init__(self):
        self.W = self.rng.normal(0, np.sqrt(2.0/(self.hidden_dim+self.hidden_dim)), (self.hidden_dim, self.hidden_dim))
        self.U = self.rng.normal(0, np.sqrt(2.0/(self.in_dim+self.hidden_dim)), (self.hidden_dim, self.in_dim))
        self.b = np.zeros((self.hidden_dim,), dtype=np.float64)
        self.V = self.rng.normal(0, np.sqrt(2.0/(self.in_dim+self.hidden_dim)), (self.hidden_dim, self.in_dim))
        self.c = self.rng.normal(0, 0.1, (self.hidden_dim,))
        self.h = np.zeros((self.hidden_dim,), dtype=np.float64)
    def reset(self): self.h[:] = 0.0
    def step(self, x: np.ndarray, energy: Optional[EnergyMeter] = None) -> np.ndarray:
        x = np.asarray(x, dtype=np.float64).reshape(-1)
        vx = self.V @ x + self.c; tau = self.tau_min + softplus(vx)
        tau = np.minimum(tau, self.tau_max)
        Wh = self.W @ self.h; Ux = self.U @ x
        a = tanh(Wh + Ux + self.b); dh = - self.h / np.maximum(tau, 1e-6) + a
        self.h = self.h + self.dt * dh
        if energy is not None: energy.add_macs((self.hidden_dim*self.hidden_dim) + (self.hidden_dim*self.in_dim))
        return self.h.copy()
    def state_dict(self) -> Dict: return {"W": self.W, "U": self.U, "b": self.b, "V": self.V, "c": self.c, "h": self.h}
    def load_state_dict(self, state: Dict):
        self.W = state["W"]; self.U = state["U"]; self.b = state["b"]
        self.V = state["V"]; self.c = state["c"]; self.h = state["h"]
@dataclass
class LiquidGatingNetwork:
    in_dim: int; hidden_dim: int; n_experts: int
    top_k: int = 2; temperature: float = 1.0
    usage_smoothing: float = 0.99; bias_lr: float = 0.01
    usage_beta: float = 0.5
    cell: LiquidCell = field(init=False); Wg: np.ndarray = field(init=False)
    bg: np.ndarray = field(init=False); usage_ma: np.ndarray = field(init=False)
    rng: np.random.Generator = field(default_factory=lambda: np.random.default_rng(2025))
    energy: EnergyMeter = field(default_factory=EnergyMeter)
    def __post_init__(self):
        self.cell = LiquidCell(self.in_dim, self.hidden_dim, rng=self.rng)
        self.Wg = self.rng.normal(0, np.sqrt(2.0/(self.hidden_dim+self.n_experts)), (self.n_experts, self.hidden_dim))
        self.bg = np.zeros((self.n_experts,), dtype=np.float64)
        self.usage_ma = np.zeros((self.n_experts,), dtype=np.float64)
        self._lock = asyncio.Lock()
    async def forward(self, x: np.ndarray, attn_gain: float = 1.0) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        async with self._lock:
            h = self.cell.step(x, self.energy)
            logits = (self.Wg @ h) + self.bg
            logits = self._apply_usage_bias(logits)
            temp = max(0.2, self.temperature / max(1e-6, attn_gain))
            probs = softmax(logits, temp=temp)
            k = max(1, min(self.top_k, self.n_experts))
            topk_idx = np.argpartition(probs, -k)[-k:]
            topk_probs = probs[topk_idx]
            if topk_probs.sum() <= 1e-12: gates = np.ones_like(topk_probs) / len(topk_probs)
            else: gates = topk_probs / topk_probs.sum()
            out = np.zeros_like(probs); out[topk_idx] = gates
            eps = 0.01
            if self.n_experts > 0 and eps > 0:
                j = int(np.argmin(self.usage_ma)); out = (1.0 - eps) * out; out[j] += eps
            self.usage_ma = self.usage_smoothing * self.usage_ma + (1.0 - self.usage_smoothing) * out
            # Track last winners for dopamine reward (from liquidmoe.py)
            self.last_winners = topk_idx
            return out, topk_idx, probs
    def _apply_usage_bias(self, logits: np.ndarray) -> np.ndarray:
        eps = 1e-6; target = 1.0 / self.n_experts
        inv_usage = target / (self.usage_ma + eps)
        return logits + self.usage_beta * np.log(inv_usage)
    async def apply_endocrine(self, *, cortisol: float = 0.0, gh: float = 0.0,
                             thyroid: float = 1.0, dopamine: float = 0.0, eps: Optional[float] = None) -> None:
        """Apply endocrine modulation to gating network (from liquidmoe.py)"""
        async with self._lock:
            # Temperature ↑ with cortisol (stress) — clamp for stability
            self.temperature = float(np.clip(self.temperature * (1.0 + 0.30 * cortisol), 0.5, 2.5))

            # Bias LR scales with thyroid (metabolic rate around 1.0 baseline)
            self.bias_lr = float(np.clip(self.bias_lr * (1.0 + 0.40 * (thyroid - 1.0)), 1e-4, 0.1))

            # Top-K capacity expands with GH (growth hormone), but never beyond n_experts
            base_top_k = getattr(self, 'base_top_k', self.top_k)
            self.base_top_k = base_top_k
            self.top_k = int(np.clip(round(base_top_k * (1.0 + 0.20 * gh)), 1, self.n_experts))

            # Dopamine: nudge most recent winners' biases (reward)
            if dopamine > 0 and hasattr(self, 'last_winners') and self.last_winners is not None:
                self.bg[self.last_winners] += 0.10 * float(dopamine)

            # Optional: exploration epsilon override
            if eps is not None:
                self.eps = float(np.clip(eps, 0.0, 0.05))
    async def nudge_for_load_balance(self) -> None:
        async with self._lock:
            if self.n_experts <= 0: return
            target = 1.0 / float(self.n_experts); delta = target - self.usage_ma
            self.bg += self.bias_lr * delta
    def reset(self): self.cell.reset(); self.usage_ma[:] = 0.0; self.energy.reset()
    def state_dict(self) -> Dict:
        return {"cell": self.cell.state_dict(), "Wg": self.Wg, "bg": self.bg, "usage_ma": self.usage_ma}
    def load_state_dict(self, state: Dict):
        self.cell.load_state_dict(state["cell"]); self.Wg = state["Wg"]; self.bg = state["bg"]; self.usage_ma = state["usage_ma"]
class NLMSExpertAdapter:
    def __init__(self, neuron: ExpertNLMSHead): self.neuron = neuron
    def predict(self, x: np.ndarray) -> float: return self.neuron.predict(x)
    async def update(self, x: np.ndarray, y_true: float, token_ids: List[int],
                     attention_bundle: Optional[Dict[str, Any]] = None) -> float:
        return await self.neuron.update(x, y_true, token_ids, attention_bundle)
    def state_dict(self) -> Dict: return self.neuron.state_dict()
    def load_state_dict(self, state: Dict): self.neuron.load_state_dict(state)
@dataclass
class LiquidMoERouter:
    experts: Dict[str, NLMSExpertAdapter]
    in_dim: int; hidden_dim: int; top_k: int = 2
    temperature: float = 1.0
    gating: LiquidGatingNetwork = field(init=False)
    bandit: BanditGating = field(init=False)
    names: List[str] = field(init=False)
    energy: EnergyMeter = field(default_factory=EnergyMeter)
    use_bandit: bool = True  # Enable bandit gating
    def __post_init__(self):
        self.names = list(self.experts.keys())
        self.gating = LiquidGatingNetwork(
            in_dim=self.in_dim, hidden_dim=self.hidden_dim,
            n_experts=len(self.names), top_k=self.top_k,
            temperature=self.temperature,
        )
        self.bandit = BanditGating(n_experts=len(self.names), exploration_factor=2.0)
    async def route(self, x: np.ndarray, attn_gain: float = 1.0) -> Dict[str, any]:
        gates_sparse, topk_idx_base, probs = await self.gating.forward(x, attn_gain=attn_gain)

        # Use bandit gating if enabled
        if self.use_bandit:
            # Convert sparse gates to dense for bandit
            gates_dense = np.zeros(len(self.names), dtype=np.float64)
            for i, idx in enumerate(topk_idx_base):
                gates_dense[int(idx)] = float(gates_sparse[i])

            # Get bandit-selected experts
            topk_idx, topk_gates = self.bandit.select_top_k(self.top_k, gates_dense)

            # Update gates_sparse and topk_idx
            gates_sparse = topk_gates
            topk_idx = topk_idx
        else:
            topk_idx = topk_idx_base

        chosen = [(self.names[i], float(gates_sparse[j])) for j, i in enumerate(topk_idx)]
        y = 0.0; per_expert: Dict[str, Dict[str, float]] = {}
        for j, i in enumerate(topk_idx):
            name = self.names[int(i)]; gate = float(gates_sparse[j])
            pred = float(self.experts[name].predict(x)); y += gate * pred
            self.energy.add_macs(self.in_dim); per_expert[name] = {"gate": gate, "pred": pred}
        return {'y_hat': float(y), 'topk': chosen, 'probs': probs, 'per_expert': per_expert,
            'energy_j': self.energy.total_j + self.gating.energy.total_j}
    async def learn(self, x: np.ndarray, token_ids: List[int], y_true: float,
                    attn_gain: float = 1.0, attention_bundle: Optional[Dict[str, Any]] = None) -> Dict[str, any]:
        out = await self.route(x, attn_gain=attn_gain); tasks = []
        expert_errors = {}  # Track errors for bandit updates

        for name, info in out['per_expert'].items():
            gate = float(info['gate']);
            if gate <= 0.0: continue
            target = float(y_true)
            pred = float(info['pred'])
            error = abs(target - pred)
            expert_errors[name] = error
            tasks.append(self.experts[name].update(x, target, token_ids, attention_bundle))

        await asyncio.gather(*tasks)

        # Update bandit with expert performance
        if self.use_bandit:
            for name, error in expert_errors.items():
                if name in self.names:
                    expert_idx = self.names.index(name)
                    self.bandit.update(expert_idx, error)

        await self.gating.nudge_for_load_balance(); return out
    def reset(self):
        self.gating.reset()
        self.energy.reset()
        if self.use_bandit:
            self.bandit.reset()
    def state_dict(self) -> Dict:
        expert_states = {name: expert.state_dict() for name, expert in self.experts.items()}
        bandit_state = {
            "total_rewards": self.bandit.total_rewards.tolist(),
            "selection_counts": self.bandit.selection_counts.tolist(),
            "total_selections": self.bandit.total_selections
        } if self.use_bandit else None
        return {"gating": self.gating.state_dict(), "experts": expert_states, "bandit": bandit_state}
    def load_state_dict(self, state: Dict):
        self.gating.load_state_dict(state["gating"])
        for name, expert_state in state["experts"].items():
            if name in self.experts: self.experts[name].load_state_dict(expert_state)
        if self.use_bandit and "bandit" in state and state["bandit"]:
            bandit_state = state["bandit"]
            self.bandit.total_rewards = np.array(bandit_state["total_rewards"])
            self.bandit.selection_counts = np.array(bandit_state["selection_counts"])
            self.bandit.total_selections = bandit_state["total_selections"]

# --- 12. "Temporal Mind" ---
class TemporalMemoryInterpolator:
    def __init__(self, epsilon: float = 1e-12):
        self.epsilon = epsilon
        print("CREATED: TemporalMemoryInterpolator (Aura 7.0 Hippocampus)")
    def interpolate(self, M0: np.ndarray, M1: np.ndarray, t: float,
                    mode: Literal['linear', 'fourier', 'hilbert', 'hamiltonian'] = 'hilbert'
                   ) -> np.ndarray:
        alpha = np.clip(t, 0.0, 1.0)
        if mode == 'linear': return (1.0 - alpha) * M0 + alpha * M1
        elif mode == 'fourier':
            F0 = np.fft.fft(M0); F1 = np.fft.fft(M1)
            F_interp = (1.0 - alpha) * F0 + alpha * F1
            return np.real(np.fft.ifft(F_interp))
        A0 = hilbert(M0, axis=0); A1 = hilbert(M1, axis=0)
        if mode == 'hilbert':
            A_interp = (1.0 - alpha) * A0 + alpha * A1
            return np.real(A_interp)
        elif mode == 'hamiltonian':
            print("WARNING: Hamiltonian mode is computationally expensive.")
            A_diff = (A1 - A0).astype(np.complex128)
            H_num = np.outer(A_diff, A_diff.T.conj())
            H_den = np.linalg.norm(A_diff)**2 + self.epsilon
            H = H_num / H_den; U = expm(-1j * H * alpha)
            A_interp = U @ A0; return np.real(A_interp)
        else: raise ValueError(f"Unknown interpolation mode: {mode}")

# --- 13. CNS & ThalamicRouter (Trained) ---
class ConsciousnessLevel(Enum):
    DEEP_SLEEP = 0; ASLEEP = 1; ALERT = 2; FOCUSED = 3; HYPERVIGILANT = 4
class CentralNervousSystem:
    def __init__(self):
        self.consciousness_level = ConsciousnessLevel.ALERT
        self.stress_level = 0.0 # 'cortisol'
        self.consolidation_factor = 1.0 # Reduces impact of errors after dreaming
        print("CREATED: CentralNervousSystem (CNS)")
    def set_consciousness(self, level: ConsciousnessLevel):
        if self.consciousness_level != level:
            self.consciousness_level = level; print(f"CNS: Consciousness set to {level.name}")
    def update_stress(self, error: float):
        new_stress = abs(error) * 1.5 * self.consolidation_factor
        self.stress_level = (self.stress_level * 0.5) + (new_stress * 0.5)
        self.stress_level = max(0.0, self.stress_level - 0.1)
        if self.stress_level > 1.0: self.set_consciousness(ConsciousnessLevel.HYPERVIGILANT)
        else: self.set_consciousness(ConsciousnessLevel.ALERT)
    def apply_consolidation(self, factor: float = 0.7):
        """Apply memory consolidation effect that reduces impact of future errors"""
        self.consolidation_factor = factor

    def get_endocrine_levels(self) -> Dict[str, float]:
        """Compute endocrine hormone levels from CNS state (for liquidmoe.py integration)"""
        # Cortisol: stress level (0-1 maps to 0-2)
        cortisol = min(2.0, self.stress_level * 2.0)

        # Growth Hormone (GH): higher when learning (low stress, alert)
        gh = 0.5 if self.consciousness_level == ConsciousnessLevel.ALERT else 0.0

        # Thyroid: metabolic rate (higher when alert, lower when stressed)
        thyroid = 1.0 - (self.stress_level * 0.3)  # 1.0 baseline, down to 0.7 when stressed

        # Dopamine: reward signal (higher when stress is low and consciousness is alert)
        dopamine = max(0.0, 1.0 - self.stress_level) if self.consciousness_level == ConsciousnessLevel.ALERT else 0.0

        return {
            'cortisol': cortisol,
            'gh': gh,
            'thyroid': thyroid,
            'dopamine': dopamine
        }
class TrainedThalamicRouter:
    """The 'School' - runs the offline-trained PyTorch model"""
    def __init__(self, model_path: str, label_maps: Dict, feature_gen: FeatureGenerator):
        self.label_maps = label_maps
        self.emotion_map = label_maps.get('emotion', {})
        self.intent_map = label_maps.get('intent', {})
        self.emotion_labels_inv = {v: k for k, v in self.emotion_map.items()}
        self.feature_gen = feature_gen; self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        try:
            self.model = MultiTaskHead(
                feature_gen.TOTAL_FEATURES,
                len(self.emotion_map), len(self.intent_map), len(self.emotion_map) # Mock tone
            ).to(self.device)
            self.model.load_state_dict(torch.load(model_path))
            self.model.eval()
            print(f"CREATED: TrainedThalamicRouter (Loaded from {model_path})")
        except Exception as e:
            print(f"--- ❌ ERROR: Failed to load ThalamicRouter model. {e} ---")

    def get_target_signals(self, query: str) -> dict:
        if not self.model: return {'emotion': 0.0, 'intent': 0.0}
        record = {"text": query, "plutchik": {"primary": "neutral"}}
        sbert_vec = self.feature_gen.sbert_model.encode(query, normalize_embeddings=True)
        # --- THE FIX: We must use use_emotion_sine=False ---
        x_raw = self.feature_gen.build_features(record, sbert_vec, use_emotion_sine=False)
        x_tensor = torch.tensor(x_raw, dtype=torch.float32).unsqueeze(0).to(self.device)
        with torch.no_grad():
            emo_logits, intent_logits, _ = self.model(x_tensor)
            emo_pred_index = emo_logits.argmax(dim=1).item()
            intent_pred_index = intent_logits.argmax(dim=1).item()
        pred_label = self.emotion_labels_inv.get(emo_pred_index, 'neutral')
        signals = {'emotion': float(emo_pred_index), 'intent': float(intent_pred_index)}
        print(f"Thalamus Predicted: '{pred_label}' (Index: {emo_pred_index})")
        return signals

# --- 14. The Final IBNN (Aura 7.0 - Hebbian-MoE-Temporal) ---
class IntegratedBioNeuralNetwork:
    """
    Aura 7.0: Integrates all components into a persistent,
    unsupervised-learning, and time-aware architecture.
    """
    def __init__(self, n_experts: int = 15, hebbian_components: int = 64,
                 llm_model_name: str = "meta-llama/Llama-3.2-3B-Instruct"):
        print("--- 🧠 Initializing Aura 7.0 (Hebbian-MoE-Temporal) ---")
        if torch.cuda.is_available():
            print(f"   -> GPU: {torch.cuda.get_device_name(0)}, VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
            maybe_empty_cuda_cache("startup", min_free_ratio=0.05)
        self.feature_gen = FeatureGenerator()
        self.response_gen = ResponseGenerator(llm_model_name)
        self.cns = CentralNervousSystem()
        self.interpolator = TemporalMemoryInterpolator()
        self.raw_feature_dim = self.feature_gen.TOTAL_FEATURES
        self.vocab_size = self.feature_gen.vocab_size
        self.abstract_feature_dim = hebbian_components

        self.whitener: OptimizedWhitener = None
        self.hippocampus: OjaLayer = None # Hebbian Cortex
        self.cortex: LiquidMoERouter = None # MoE Cortex
        self.router: Optional[TrainedThalamicRouter] = None
        self.n_experts = n_experts
        self.news_pattern_analyzer = None
        self.timeline_analyzer = None
        self.news_ingestion_service = None

        self.education_dir = SAVE_DIR # Use global
        os.makedirs(self.education_dir, exist_ok=True)
        print("--- Aura 7.0 Brain Initialized (Awaiting Education) ---")

    def _create_brain_layers(self, hebbian_k: int):
        """Dynamically creates the brain layers based on the Hebbian cortex size."""
        self.abstract_feature_dim = hebbian_k
        print(f"--- 🧠 Building Dynamic Brain Layers ---")
        print(f"   -> Hebbian Cortex (Oja) K = {hebbian_k}")

        self.hippocampus = OjaLayer(
            dim=self.raw_feature_dim,
            n_components=hebbian_k,
            mode='nonlinear', lr=5e-3, max_components=128, grow_threshold=0.3
        )
        self.whitener = OptimizedWhitener(dim=self.raw_feature_dim)

        experts: Dict[str, NLMSExpertAdapter] = {}
        attention_config = {
            'decay': 0.7,
            'theta': 1.0,
            'k_winners': 3,
            'gain_up': 1.5,
            'gain_down': 0.7,
            'multi_channel': {
                'k_winners': 5,
                'w_amp': 1.0,
                'w_pitch': 1.4,
                'w_bound': 0.8,
                'gain_up': 2.0,
                'gain_down': 0.5,
                'use_stdp': True,
                'smoothing': 1,
            }
        }
        num_emotions = len(GOEMOTIONS_LABELS)
        emotion_range_per_expert = num_emotions / self.n_experts

        for i in range(self.n_experts):
            name = f"expert__{i}"
            # Initialize bias toward expert's emotion range (0-27 for GoEmotions)
            target_emotion_idx = i * emotion_range_per_expert + emotion_range_per_expert / 2
            head = ExpertNLMSHead(
                n_features=self.abstract_feature_dim,
                vocab_size=self.vocab_size,
                attention_config=attention_config,
                mu=0.5,  # Even higher learning rate for faster convergence
                mu_decay=0.99995,  # Very slow decay to maintain learning longer
                mu_min=0.1,  # Higher minimum learning rate
                initial_bias=target_emotion_idx  # Initialize bias to target range
            )
            # Initialize weights with larger random values for better coverage
            head.w = np.random.normal(0, 1.0, self.abstract_feature_dim)
            # Ensure bias is actually set (double-check)
            head.bias = float(target_emotion_idx)
            experts[name] = NLMSExpertAdapter(neuron=head)

        self.cortex = LiquidMoERouter(
            experts=experts, in_dim=self.abstract_feature_dim, # Use K
            hidden_dim=128, top_k=3
        )
        print(f"--- 🧠 Dynamic Brain Layers Created ---")

    async def educate_brain(self):
        """
        Runs the 'School' pipeline to pre-train all components.

        """
        print("\n--- 🎓 STARTING OFFLINE EDUCATION (GoEmotions) ---")
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # 1. Load GoEmotions
        print("Loading 'google-research-datasets/go_emotions'...")
        dataset = load_dataset("google-research-datasets/go_emotions", split='train')
        dataset = dataset.shuffle(seed=42).select(range(5000))
        all_texts = dataset['text']

        # 2. Precompute SBERT
        print(f"Pre-computing SBERT embeddings for {len(all_texts)} texts...")
        sbert_embeddings = await asyncio.to_thread(
            self.feature_gen.sbert_model.encode,
            all_texts, normalize_embeddings=True, batch_size=256
        )

        # 3. Create Label Maps
        emotion_labels = GOEMOTIONS_LABELS
        intent_labels = ["question", "statement", "exclamation", "request", "none"]
        tone_labels = emotion_labels # Mock
        label_maps = {
            'emotion': {label: idx for idx, label in enumerate(emotion_labels)},
            'intent': {label: idx for idx, label in enumerate(intent_labels)},
            'tone': {label: idx for idx, label in enumerate(tone_labels)},
        }

        # 4. Build Features
        print("Building features...")
        X_features, y_emotion, y_intent, y_tone = [], [], [], []
        for i, record in enumerate(dataset):
            primary_emotion = 'neutral'
            label_indices = record['labels']
            if label_indices:
                for idx in label_indices:
                    if 0 <= idx < len(GOEMOTIONS_LABELS):
                        label_name = GOEMOTIONS_LABELS[idx]
                        if label_name != 'neutral':
                            primary_emotion = label_name; break

            mock_record = {
                "text": record['text'],
                "plutchik": {"primary": primary_emotion, "intensity": 1.0},
                "intent": "question" if "?" in record['text'] else "statement",
                "tone": primary_emotion
            }
            # --- THE FIX: Train Thalamus on "neutral" sine wave ---
            X_features.append(self.feature_gen.build_features(mock_record, sbert_embeddings[i], use_emotion_sine=False))
            y_emotion.append(label_maps['emotion'].get(primary_emotion, GOEMOTIONS_MAP['neutral']))
            y_intent.append(label_maps['intent'].get(mock_record['intent'], 1))
            y_tone.append(label_maps['tone'].get(primary_emotion, GOEMOTIONS_MAP['neutral']))

        X_train_np = np.stack(X_features)
        y_emotion_np = np.array(y_emotion)

        indices = np.arange(len(X_train_np))
        X_train_split, X_test_split, y_emotion_train_split, y_emotion_test_split, \
        y_intent_train_split, y_intent_test_split, y_tone_train_split, y_tone_test_split = \
            train_test_split(X_train_np, y_emotion_np, y_intent, y_tone, test_size=0.2, random_state=42, stratify=y_emotion_np)

        X_train_torch = torch.tensor(X_train_split, dtype=torch.float32).to(device)
        y_emotion_train_torch = torch.tensor(y_emotion_train_split, dtype=torch.long).to(device)
        y_intent_train_torch = torch.tensor(y_intent_train_split, dtype=torch.long).to(device)
        y_tone_train_torch = torch.tensor(y_tone_train_split, dtype=torch.long).to(device)

        X_test_torch = torch.tensor(X_test_split, dtype=torch.float32).to(device)
        y_emotion_test_torch = torch.tensor(y_emotion_test_split, dtype=torch.long).to(device)
        y_intent_test_torch = torch.tensor(y_intent_test_split, dtype=torch.long).to(device)
        y_tone_test_torch = torch.tensor(y_tone_test_split, dtype=torch.long).to(device)

        # --- 5. Train the "Hebbian Cortex" (OjaLayer) ---
        print("Training Hebbian Cortex (OjaLayer) on whitened data...")
        self._create_brain_layers(hebbian_k=self.abstract_feature_dim)

        # Note: We train Oja on the *real* features, with sine waves
        print("Building full features for Hebbian training...")
        X_features_hebbian = []
        for i, record in enumerate(dataset): # Using the full dataset
            primary_emotion = 'neutral'
            label_indices = record['labels']
            if label_indices:
                for idx in label_indices:
                    if 0 <= idx < len(GOEMOTIONS_LABELS):
                        label_name = GOEMOTIONS_LABELS[idx]
                        if label_name != 'neutral':
                            primary_emotion = label_name; break
            mock_record = {"text": record['text'], "plutchik": {"primary": primary_emotion, "intensity": 1.0}}
            # We use use_emotion_sine=True (default) for Hebbian layer
            X_features_hebbian.append(self.feature_gen.build_features(mock_record, sbert_embeddings[i], use_emotion_sine=True))

        X_train_np_hebbian = np.stack(X_features_hebbian)
        X_features_whitened = np.array([self.feature_gen.whitener.transform(x) for x in X_train_np_hebbian])

        for x_w in X_features_whitened:
             self.hippocampus.step(x_w)

        final_k = self.hippocampus.K
        if final_k != self.abstract_feature_dim:
            print(f"Hebbian Cortex grew from {self.abstract_feature_dim} to {final_k}. Rebuilding MoE...")
            self._create_brain_layers(hebbian_k=final_k)

        print(f"   -> Hebbian Cortex (OjaLayer) training complete. Final K={final_k}")

        # --- 6. Train the "Thalamus" (MultiTaskHead) ---
        print(f"Training MultiTask ThalamicRouter...")
        class_weights_emotion = compute_class_weight('balanced', classes=np.unique(y_emotion_train_split), y=y_emotion_train_split)
        class_weights_emotion = torch.tensor(class_weights_emotion, dtype=torch.float32).to(device)

        thalamus_model = MultiTaskHead(
            self.feature_gen.TOTAL_FEATURES,
            len(label_maps['emotion']), len(label_maps['intent']), len(label_maps['tone'])
        ).to(device)

        optimizer = optim.AdamW(thalamus_model.parameters(), lr=1e-3)
        criterion_emotion = nn.CrossEntropyLoss(weight=class_weights_emotion)
        criterion_intent = nn.CrossEntropyLoss()
        criterion_tone = nn.CrossEntropyLoss(weight=class_weights_emotion)

        best_val_loss = float('inf')
        epochs_no_improve = 0
        patience = 100

        for epoch in range(300):
            thalamus_model.train()
            optimizer.zero_grad()
            emo_out, intent_out, tone_out = thalamus_model(X_train_torch)
            loss = (1.0 * criterion_emotion(emo_out, y_emotion_train_torch) +
                    0.5 * criterion_intent(intent_out, y_intent_train_torch) +
                    0.5 * criterion_tone(tone_out, y_tone_train_torch))
            loss.backward(); optimizer.step()

            thalamus_model.eval()
            with torch.no_grad():
                emo_val, intent_val, tone_val = thalamus_model(X_test_torch)
                val_loss = (1.0 * criterion_emotion(emo_val, y_emotion_test_torch) +
                            0.5 * criterion_intent(intent_val, y_intent_test_torch) +
                            0.5 * criterion_tone(tone_val, y_tone_test_torch))

            print(f"Epoch {epoch+1:02d}/30 | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 1
                thalamus_path = os.path.join(self.education_dir, "thalamic_router_multitask.pt")
                torch.save(thalamus_model.state_dict(), thalamus_path)
                print(f"   -> New best model saved to {thalamus_path}")
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"   -> Early stopping on epoch {epoch+1}.")
                    break

        # --- 7. "Mind Upload" - Initialize Router ---
        self.router = TrainedThalamicRouter(thalamus_path, label_maps, self.feature_gen)
        print("--- 🎓 OFFLINE EDUCATION COMPLETE ---")

    async def process_query(self, query: str, update_stress: bool = True) -> str:
        print(f"\n--- ☀️ PROCESSING QUERY (Aura 7.0): '{query}' ---")
        if not self.router: return "I am uneducated. Please run `await aura.educate_brain()`."

        cns_level = self.cns.consciousness_level

        # 1. "Ears" -> "Hebbian Cortex"
        # The Hebbian layer *can* see the emotional sine wave, as it's part of the raw sensory context
        x_raw, token_ids, attention_bundle = self.feature_gen.generate_for_query(query)
        xw = self.feature_gen.whitener.transform(x_raw)
        oja_out = self.hippocampus.step(xw)
        y_abstract = oja_out.y # The abstract "engram"

        # 2. "Thalamus" (Educated)
        # The Thalamus *must not* see the sine wave it's trying to predict.
        target_signals = self.router.get_target_signals(query)
        y_true_emotion_idx = target_signals['emotion']

        # The MoE experts learn to map the *abstract engram* to the *Thalamus's target*
        target_for_moe = float(y_true_emotion_idx)

        out = await self.cortex.learn(
            x=y_abstract, token_ids=token_ids, y_true=target_for_moe, attention_bundle=attention_bundle
        )

        # 4. "CNS" (Feeling)
        raw_prediction = out['y_hat']
        moe_idx, discrete_prediction = quantize_moe_prediction(raw_prediction, NUM_GOEMOTIONS)
        prediction_error = target_for_moe - discrete_prediction
        if update_stress:
            self.cns.update_stress(prediction_error)
            # Apply endocrine modulation to gating network (from liquidmoe.py)
            endocrine_levels = self.cns.get_endocrine_levels()
            await self.cortex.gating.apply_endocrine(**endocrine_levels)

        print(f"Hebbian (Oja) Residual: {oja_out.residual_ema:.3f} (Grew: {oja_out.grew})")
        print(f"MoE Prediction: idx={moe_idx} (raw={raw_prediction:.2f}), Target: {int(target_for_moe)}, Error: {abs(prediction_error):.2f}")

        # Diagnostic: Show top expert contributions
        if 'topk' in out and len(out['topk']) > 0:
            top_expert = out['topk'][0]
            expert_name, expert_gate = top_expert
            expert_neuron = self.cortex.experts[expert_name].neuron
            expert_rmse = expert_neuron.get_rmse()
            expert_lr = expert_neuron.mu
            print(f"   -> Top Expert: {expert_name} (gate={expert_gate:.3f}, RMSE={expert_rmse:.2f}, LR={expert_lr:.4f})")

        # 5. "Mouth" speaks
        final_brain_state = {
            'cns_state': self.cns.consciousness_level,
            'stress_level': self.cns.stress_level,
        }
        response = await self.response_gen.generate_response(query, final_brain_state)
        self.last_run_final_stress = self.cns.stress_level
        return response

    async def process_live_news(self, articles: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Ingest news articles and optionally update pattern analyzers."""
        if not articles:
            return []
        if not self.router or not self.cortex:
            print("[WARN] Cannot process news articles before education.")
            return []
        if self.news_pattern_analyzer is None:
            self.news_pattern_analyzer = NewsPatternAnalyzer()
        if self.news_ingestion_service is None:
            self.news_ingestion_service = NewsIngestionService(pattern_analyzer=self.news_pattern_analyzer)
        return await self.news_ingestion_service.process_articles(self, articles)

    def summarize_news_trends(self, topic: str = "general") -> Dict[str, Any]:
        if not self.news_pattern_analyzer:
            return {}
        return self.news_pattern_analyzer.summarize_topic(topic)

    def register_historical_timeline(self, entity: str, events: List[Dict[str, Any]]):
        if self.timeline_analyzer is None:
            self.timeline_analyzer = HistoricalTimelineAnalyzer()
        self.timeline_analyzer.register_entity(entity, events)

    def compare_historical_pattern(self, query: str, window: int = 5) -> Dict[str, Any]:
        if not self.timeline_analyzer:
            return {}
        return self.timeline_analyzer.compare_pattern(query, window=window)

    def get_moe_statistics(self) -> Dict[str, Any]:
        """Get diagnostic statistics about MoE experts"""
        stats = {
            'expert_count': len(self.cortex.experts),
            'experts': {}
        }
        for name, expert_adapter in self.cortex.experts.items():
            neuron = expert_adapter.neuron
            stats['experts'][name] = {
                'rmse': neuron.get_rmse(),
                'learning_rate': neuron.mu,
                'update_count': neuron.update_count,
                'last_error': neuron.last_error
            }
        return stats

    def _get_brain_state_vector(self) -> np.ndarray:
        all_weights = []
        all_weights.append(self.feature_gen.whitener.mu.flatten())
        all_weights.append(self.feature_gen.whitener.var.flatten())
        all_weights.append(self.hippocampus.W.flatten())
        all_weights.append(self.cortex.gating.cell.W.flatten())
        all_weights.append(self.cortex.gating.cell.U.flatten())
        all_weights.append(self.cortex.gating.cell.b.flatten())
        all_weights.append(self.cortex.gating.cell.V.flatten())
        all_weights.append(self.cortex.gating.cell.c.flatten())
        all_weights.append(self.cortex.gating.Wg.flatten())
        all_weights.append(self.cortex.gating.bg.flatten())
        for expert in self.cortex.experts.values():
            all_weights.append(expert.neuron.w.flatten())
        return np.concatenate(all_weights)

    def _load_brain_state_vector(self, M: np.ndarray):
        print("--- 🧠 Loading brain state from vector... ---")
        idx = 0
        def _load(target_array: np.ndarray) -> int:
            nonlocal idx
            n = target_array.size
            try:
                target_array[:] = M[idx : idx + n].reshape(target_array.shape)
            except ValueError:
                print(f"!! Shape mismatch during load. Target: {target_array.shape}, Got: {M[idx : idx + n].shape}")
                min_n = min(target_array.size, M[idx:].size)
                if min_n == 0: return idx
                target_flat = target_array.flat; source_flat = M[idx:].flat
                for i in range(min_n): target_flat[i] = source_flat[i]
                n = min_n
            return idx + n
        try:
            idx = _load(self.feature_gen.whitener.mu)
            idx = _load(self.feature_gen.whitener.var)
            idx = _load(self.hippocampus.W)
            idx = _load(self.cortex.gating.cell.W)
            idx = _load(self.cortex.gating.cell.U)
            idx = _load(self.cortex.gating.cell.b)
            idx = _load(self.cortex.gating.cell.V)
            idx = _load(self.cortex.gating.cell.c)
            idx = _load(self.cortex.gating.Wg)
            idx = _load(self.cortex.gating.bg)
            for expert in self.cortex.experts.values():
                idx = _load(expert.neuron.w)
            print(f"--- 🧠 Brain state loaded successfully. Total params: {idx} ---")
        except Exception as e:
            print(f"--- ❌ ERROR: Failed to load brain state vector. {e} ---")

    async def save_keyframe(self, name: str):
        path = os.path.join(self.education_dir, f"keyframe_{name}_{int(time.time())}.npy")
        print(f"\n--- 💾 Saving keyframe to {path} ---")
        M = self._get_brain_state_vector()
        await asyncio.to_thread(np.save, path, M)
        print("--- 💾 Keyframe save complete ---")

    async def run_sleep_cycle(self, keyframe1_name: str, keyframe2_name: str):
        """
        Implements "Dream Recall" from the paper.
        Consolidates memories and reduces stress through dream interpolation.
        """
        print(f"\n--- 💤 RUNNING SLEEP CYCLE (Dreaming) ---")
        print(f"Loading keyframes: {keyframe1_name} and {keyframe2_name}")
        try:
            M0_path = sorted(glob.glob(os.path.join(self.education_dir, f"keyframe_{keyframe1_name}*.npy")))[-1]
            M1_path = sorted(glob.glob(os.path.join(self.education_dir, f"keyframe_{keyframe2_name}*.npy")))[-1]
            M0 = np.load(M0_path)
            M1 = np.load(M1_path)
        except Exception as e:
            print(f"--- ❌ ERROR: Could not load keyframes for dreaming. {e} ---"); return

        current_brain_shape = self._get_brain_state_vector().shape
        if M0.shape != M1.shape or M0.shape != current_brain_shape:
            print(f"--- ❌ ERROR: Keyframe dimension mismatch. Cannot interpolate. ---")
            print(f"M0: {M0.shape}, M1: {M1.shape}, Current: {current_brain_shape}"); return

        stress_before_dream = self.cns.stress_level

        print("...Interpolating dream state using Hilbert (phase-aware) interpolation...")
        # Use multiple interpolation points for richer consolidation
        interpolation_points = [0.25, 0.5, 0.75]
        M_dreams = []
        for t in interpolation_points:
            M_dream = self.interpolator.interpolate(M0, M1, t=t, mode='hilbert')
            M_dreams.append(M_dream)

        # Average the interpolated states for smoother consolidation
        M_consolidated = np.mean(M_dreams, axis=0)
        self._load_brain_state_vector(M_consolidated)

        print("...Consolidating dream state (multiple interpolation points)...")
        # Process multiple dream queries for better consolidation
        dream_queries = ["... (dreaming) ...", "... (memory consolidation) ...", "... (neural replay) ..."]
        for dream_query in dream_queries:
            await self.process_query(dream_query, update_stress=False)

        stress_after_consolidation = self.cns.stress_level
        stress_reduction_factor = 0.25
        stress_reduction = max(0.0, stress_before_dream * stress_reduction_factor)
        self.cns.stress_level = max(0.0, stress_after_consolidation - stress_reduction)

        self.cns.apply_consolidation(factor=0.7)

        if self.cns.stress_level < 1.0:
            self.cns.set_consciousness(ConsciousnessLevel.ALERT)

        print(f"--- 💤 SLEEP CYCLE COMPLETE (Stress reduced by {stress_reduction:.4f}, from {stress_before_dream:.4f} to {self.cns.stress_level:.4f}, consolidation factor: {self.cns.consolidation_factor:.2f}) ---")

# --- 15. PyTorch Models (for the "School") ---
class LinearTorchModel(nn.Module):
    """"""
    def __init__(self, input_dim: int, num_classes: int):
        super().__init__(); self.fc = nn.Linear(input_dim, num_classes)
    def forward(self, x): return self.fc(x)
# --- All Imports ---
import uuid
import enum
from enum import Enum # Explicit import
import numpy as np
import random
import json
import itertools
import torch
from transformers import AutoTokenizer, logging
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import os
from unsloth import FastLanguageModel
from typing import List, Dict, Optional, Union, Any, Tuple, Literal # Explicit imports
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from numpy import ndarray
import asyncio
import io
import csv
import threading # For Memory Pool
import time
from scipy.signal import hilbert # For Temporal Interpolator
from scipy.linalg import expm # For Temporal Interpolator
import glob # For finding latest keyframe
from datasets import load_dataset # For GoEmotions

# --- PyTorch Imports (for Pre-Training) ---
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# Suppress transformer warnings
logging.set_verbosity_error()

# --- 0. Colab/HF Login ---
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("Hugging Face login successful!")
except (ImportError, KeyError):
    print("HF_TOKEN secret not found or not in Colab. Ensure you are logged in.")

# --- 1. Helper Functions (Softplus, Tanh, Softmax) ---
def softplus(z: np.ndarray) -> np.ndarray:
    return np.log1p(np.exp(-np.abs(z))) + np.maximum(z, 0)
def tanh(z: np.ndarray) -> np.ndarray:
    return np.tanh(z)
def softmax(z: np.ndarray, temp: float = 1.0) -> np.ndarray:
    z = z / max(1e-8, temp); z = z - np.max(z); ez = np.exp(z)
    s = ez.sum(); return ez / (s + 1e-12)

# --- 2. EnergyMeter ---
@dataclass
class EnergyMeter:
    e_mac_j: float = 3e-12; total_j: float = 0.0
    def add_macs(self, nmacs: int): self.total_j += self.e_mac_j * float(nmacs)

# --- 3. Memory Pool (from memory_pool.py) ---
@dataclass
class PoolStats:
    hits: int = 0; misses: int = 0; total_allocations: int = 0; peak_usage_mb: float = 0.0
class ArrayPool:
    """"""
    def __init__(self, max_pool_size_mb: int = 512):
        self.max_pool_size = max_pool_size_mb * 1024 * 1024
        self.pools: Dict[Tuple[tuple, np.dtype], List[np.ndarray]] = {}
        self.current_usage = 0; self.stats = PoolStats(); self._lock = threading.Lock()
    def get_array(self, shape: tuple, dtype: np.dtype = np.float32, zero_fill: bool = True) -> np.ndarray:
        key = (shape, dtype)
        with self._lock:
            if key in self.pools and self.pools[key]:
                arr = self.pools[key].pop(); self.stats.hits += 1
                if zero_fill: arr.fill(0); return arr
            else:
                arr = np.empty(shape, dtype=dtype);
                if zero_fill: arr.fill(0)
                self.stats.misses += 1; self.stats.total_allocations += 1; return arr
    def return_array(self, arr: np.ndarray) -> None:
        if arr is None: return
        key = (arr.shape, arr.dtype); array_size = arr.nbytes
        with self._lock:
            if self.current_usage + array_size <= self.max_pool_size:
                if key not in self.pools: self.pools[key] = []
                arr.fill(0); self.pools[key].append(arr); self.current_usage += array_size
_global_pool = ArrayPool()
def get_pooled_array(shape: tuple, dtype: np.dtype = np.float32, zero_fill: bool = True) -> np.ndarray:
    return _global_pool.get_array(shape, dtype, zero_fill)
def return_pooled_array(arr: np.ndarray) -> None:
    _global_pool.return_array(arr)

# --- 4. Optimized Whitener (from training_coordinator_optimized.py) ---
class OptimizedWhitener:
    """"""
    def __init__(self, dim: int, eps: float = 1e-6, momentum: float = 0.01):
        self.dim = dim; self.eps = np.float32(eps); self.momentum = np.float32(momentum)
        self.mu = np.zeros(dim, dtype=np.float32); self.var = np.ones(dim, dtype=np.float32)
        self._temp_diff = np.zeros(dim, dtype=np.float32)
        self._temp_result = np.zeros(dim, dtype=np.float32)
    def transform(self, x: np.ndarray) -> np.ndarray:
        if x.dtype != np.float32: x = x.astype(np.float32)
        self.mu *= (1.0 - self.momentum); self.mu += self.momentum * x
        np.subtract(x, self.mu, out=self._temp_diff)
        np.multiply(self._temp_diff, self._temp_diff, out=self._temp_result)
        self.var *= (1.0 - self.momentum); self.var += self.momentum * self._temp_result
        np.sqrt(self.var + self.eps, out=self._temp_result)
        np.divide(self._temp_diff, self._temp_result, out=self._temp_result)
        return self._temp_result.copy()
    def state_dict(self) -> Dict: return {"mu": self.mu, "var": self.var}
    def load_state_dict(self, state: Dict): self.mu = state["mu"]; self.var = state["var"]

# --- 5. GoEmotions Labels (The Curriculum) ---
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
GOEMOTIONS_MAP = {label: idx for idx, label in enumerate(GOEMOTIONS_LABELS)}

# --- 6. "Ears": FeatureGenerator (GoEmotions-Aware) ---
class FeatureGenerator:
    """Based on clean_amygdala_trainer.py"""
    def __init__(self, sbert_model_name: str = "all-MiniLM-L6-v2"):
        self.SBERT_DIM = 384; self.SINE_LENGTH = 32; self.EXTRA_FEATURES = 3
        self.TOTAL_FEATURES = self.SBERT_DIM + self.SINE_LENGTH + self.EXTRA_FEATURES # 419
        print(f"CREATED: FeatureGenerator (Aura 7.0 Ears), Features: {self.TOTAL_FEATURES}")
        self.sbert_model = SentenceTransformer(sbert_model_name, device="cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = self.sbert_model.tokenizer
        self.vocab_size = self.tokenizer.vocab_size
        self.label_params = self._generate_default_params(GOEMOTIONS_LABELS)
        self.whitener = OptimizedWhitener(dim=self.TOTAL_FEATURES)

    def _generate_default_params(self, labels: List[str]) -> Dict[str, Dict]:
        params = {}
        for idx, label in enumerate(labels):
            params[label] = {"freq": 1.5 + 0.1 * idx, "amp": 0.7, "phase": 0.5 + 0.2 * idx}
        return params

    def build_features(self, record: Dict[str, Any], sbert_vec: np.ndarray, use_emotion_sine: bool = True) -> np.ndarray:
        """
        Build fused features.
        use_emotion_sine=False forces a neutral sine wave, preventing data leakage.
        """
        if use_emotion_sine:
            prim = record.get("plutchik", {}).get("primary", "neutral")
        else:
            prim = "neutral" # <-- THE FIX

        cfg = self.label_params.get(prim, {"freq": 1.0, "amp": 0.0, "phase": 0.0})
        t = np.linspace(0, 2*np.pi, self.SINE_LENGTH, dtype=np.float32)
        emb = (cfg["amp"] * np.sin(cfg["freq"] * t + cfg["phase"])).astype(np.float32)
        text = record.get("text", "")
        extras = np.array([len(text) / 100.0, int("!" in text), int("?" in text)], dtype=np.float32)
        return np.concatenate([emb, extras, sbert_vec]).astype(np.float32)

    def _build_attention_bundle(self, text: str) -> Dict[str, Any]:
        tokens = simple_tokenize_for_attention(text.lower())
        amp, pitch, boundary = prosody_channels_from_text(tokens)
        token_ids = [hash(tok) % self.vocab_size for tok in tokens]
        return {
            'token_ids': token_ids,
            'amp': amp,
            'pitch': pitch,
            'boundary': boundary,
            'raw_tokens': tokens,
        }

    def generate_for_query(self, query: str, thalamus_prediction: str = 'neutral') -> Tuple[np.ndarray, List[int], Dict[str, Any]]:
        """
        Generates feature vector for a LIVE query.
        It now uses the Thalamus's prediction to build the sine wave.
        """
        record = {"text": query, "plutchik": {"primary": thalamus_prediction}}
        sbert_vec = self.sbert_model.encode(query, normalize_embeddings=True)
        # We use use_emotion_sine=True (default) for the live brain
        x_raw = self.build_features(record, sbert_vec, use_emotion_sine=True)
        x_whitened = self.whitener.transform(x_raw)
        attention_bundle = self._build_attention_bundle(query)
        token_ids = attention_bundle['token_ids']
        return x_whitened, token_ids, attention_bundle

    def state_dict(self) -> Dict: return {"whitener_mu": self.whitener.mu, "whitener_var": self.whitener.var}
    def load_state_dict(self, state: Dict): self.whitener.mu = state["mu"]; self.whitener.var = state["var"]

# --- 7. "Mouth": ResponseGenerator (Unsloth, Async) ---
class ResponseGenerator:
    def __init__(self, model_name: str = "meta-llama/Llama-3.2-3B-Instruct"):
        print(f"CREATED: ResponseGenerator (Mouth) using Unsloth on '{model_name}'")
        max_seq_length = 2048; dtype = None; load_in_4bit = True
        unsloth_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
        print(f"   -> Using optimized Unsloth 4-bit model: {unsloth_model_name}")
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name = unsloth_model_name, max_seq_length = max_seq_length,
            dtype = dtype, load_in_4bit = load_in_4bit,
        )
        if self.tokenizer.pad_token is None: self.tokenizer.pad_token = self.tokenizer.eos_token
        print("   -> Unsloth Llama 3.2 model loaded successfully.")
    def _build_prompt(self, user_query: str, brain_state: dict) -> str:
        cns_state = brain_state.get('cns_state', 'ALERT').name
        stress = brain_state.get('stress_level', 0.0)
        persona = f"You are Aura, a bio-neural AI. Your current internal state is {cns_state}."
        if cns_state == 'HYPERVIGILANT' or stress > 1.0:
            persona += f" You are feeling a high-stress level ({stress:.2f})."
        else: persona += " You are calm and helpful."
        messages = [{"role": "system", "content": persona}, {"role": "user", "content": user_query}]
        return self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    async def generate_response(self, user_query: str, brain_state: dict) -> str:
        print(f"\n--- 👄 ResponseGenerator (Unsloth) ---"); print(f"Generating response. Brain state: {brain_state}")
        prompt = self._build_prompt(user_query, brain_state)
        inputs = self.tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")
        terminators = [self.tokenizer.eos_token_id, self.tokenizer.convert_tokens_to_ids("<|eot_id|>")]
        outputs = await asyncio.to_thread(
            self.model.generate, **inputs, max_new_tokens=150, eos_token_id=terminators,
            do_sample=True, temperature=0.7, top_p=0.9,
        )
        response_text = self.tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[-1]:], skip_special_tokens=True)[0]
        print(f"Llama 3.2 (Unsloth) Output: {response_text}"); return response_text

# --- 8. SpikingAttention (k-WTA) ---
@dataclass
class SpikingAttention:
    decay: float = 0.7; theta: float = 1.0; k_winners: int = 5
    gain_up: float = 1.5; gain_down: float = 0.6
    def compute_gains(self, token_seq: List[int], vocab_size: int) -> Optional[np.ndarray]:
        if not token_seq: return None
        v: Dict[int, float] = {}; spikes: Dict[int, int] = {}
        for j in token_seq:
            vj = self.decay * v.get(j, 0.0) + 1.0
            if vj >= self.theta: spikes[j] = spikes.get(j, 0) + 1; vj -= self.theta
            v[j] = vj
        ranked = sorted(spikes.items(), key=lambda kv: (-kv[1], -v.get(kv[0], 0.0)))
        winners = set(j for j,_ in ranked[:max(1, self.k_winners)])
        gains = np.ones(vocab_size, dtype=np.float64)
        seen = set(spikes.keys()) | set(v.keys())
        for j in seen:
            if 0 <= j < vocab_size: gains[j] = self.gain_up if j in winners else self.gain_down
        return gains

# --- 9. Hebbian Layer (Oja) ---
@dataclass
class OjaStepOut:
    y: np.ndarray; residual_ema: float; grew: bool
class OjaLayer:
    def __init__(self, dim: int, n_components: int = 8, lr: float = 5e-4, mode: str = "nonlinear",
                 *, max_components: int = 64, lateral_beta: float = 0.05,
                 grow_threshold: float = 0.35, ema: float = 0.01, grow_cooldown: int = 100,
                 seed: Optional[int] = 1337):
        self.rng = np.random.default_rng(seed)
        self.dim = int(dim); self.lr = float(lr); self.mode = str(mode)
        self.max_components = int(max_components); self.beta = float(lateral_beta)
        self.grow_threshold = float(grow_threshold); self.ema = float(ema)
        self.grow_cooldown = int(grow_cooldown); self.cooldown = 0
        W0 = self.rng.normal(0, 0.1, (n_components, self.dim))
        self.W = (W0.T / (np.linalg.norm(W0, axis=1) + 1e-12)).T
        self.K = self.W.shape[0]; self.residual_ema = 0.0; self._steps = 0
        print(f"CREATED: OjaLayer (Hebbian Cortex) with {self.K} components, mode='{self.mode}'")
    def step(self, xw: np.ndarray) -> OjaStepOut:
        x = np.asarray(xw, dtype=np.float64); y = self.W @ x; x_hat = self.W.T @ y
        xn = float(np.dot(x, x) + 1e-12); explained = float(np.dot(x_hat, x_hat) / xn)
        residual = float(1.0 - explained)
        if self.mode == "nonlinear":
            g = y ** 3; dW = (g[:, None] * x[None, :]) - self.W
        else: # sanger
            proj = np.zeros_like(x); dW = np.zeros_like(self.W)
            for i in range(self.K):
                proj = proj + y[i] * self.W[i]; dW[i] = y[i] * (x - proj)
        YW = y @ self.W; cross = YW[None, :] - (y[:, None] * self.W)
        dW -= self.beta * (y[:, None] * cross); self.W += self.lr * dW; self._renorm_rows()
        self.residual_ema = (1.0 - self.ema) * self.residual_ema + self.ema * residual
        grew, _ = self._maybe_grow(x, x_hat); self._steps += 1
        return OjaStepOut(y=y, residual_ema=self.residual_ema, grew=grew)
    def _renorm_rows(self): self.W = (self.W.T / (np.linalg.norm(self.W, axis=1) + 1e-12)).T
    def _maybe_grow(self, x: np.ndarray, x_hat: np.ndarray) -> Tuple[bool, Optional[int]]:
        if self.cooldown > 0: self.cooldown -= 1; return False, None
        if self.K >= self.max_components: return False, None
        if self.residual_ema < self.grow_threshold: return False, None
        r = x - x_hat; nr = float(np.linalg.norm(r))
        if nr > 1e-9: w_new = r / nr
        else:
            w_new = self.rng.normal(0, 0.1, size=self.dim)
            w_new /= (np.linalg.norm(w_new) + 1e-12)
        self.W = np.vstack([self.W, w_new]); self.K += 1
        self.cooldown = self.grow_cooldown
        print(f"🧬 OJA NEUROGENESIS: Residual high. Growing new component. K={self.K}"); return True, self.K - 1
    def state_dict(self) -> Dict: return {"W": self.W, "residual_ema": self.residual_ema, "_steps": self._steps}
    def load_state_dict(self, state: Dict):
        self.W = state["W"]; self.K = state["W"].shape[0]; self.residual_ema = state["residual_ema"]; self._steps = state["_steps"]

# --- 10. "Expert Neuron": ExpertNLMSHead ---
class ExpertNLMSHead:
    def __init__(self, n_features: int, vocab_size: int,
                 attention_config: Dict, mu: float = 0.1, epsilon: float = 1e-6,
                 mu_decay: float = 0.9995, mu_min: float = 0.01, initial_bias: float = 0.0,
                 guard_sign: bool = False, error_thresh: float = 0.08,
                 arousal_band: Optional[tuple] = None):
        self.n_features = n_features
        self.w = np.zeros(self.n_features, dtype=np.float64)
        self.mu_initial = mu
        self.mu = mu
        self.mu_decay = mu_decay
        self.mu_min = mu_min
        self.epsilon = epsilon
        self.vocab_size = vocab_size
        spiking_cfg = {k: v for k, v in attention_config.items() if k in ('decay', 'theta', 'k_winners', 'gain_up', 'gain_down')}
        if not spiking_cfg:
            spiking_cfg = {'decay': 0.7, 'theta': 1.0, 'k_winners': 5, 'gain_up': 1.5, 'gain_down': 0.6}
        self.spiking_attention = SpikingAttention(**spiking_cfg)
        multi_cfg = attention_config.get('multi_channel') if isinstance(attention_config, dict) else None
        self.multi_attention = MultiChannelSpikingAttention(**multi_cfg) if multi_cfg else None
        self.last_error = 0.0
        self.last_output = 0.0
        self.update_count = 0
        self.total_error_sq = 0.0
        self._lock = asyncio.Lock()
        self.bias = initial_bias  # Add bias term for better learning
        # Guard conditions from nlms.py
        self.guard_sign = guard_sign  # Skip update if sign mismatch but error small
        self.error_thresh = error_thresh  # Threshold for guard conditions
        self.arousal_band = arousal_band  # (lo, hi) band for arousal-based skipping
    def predict(self, x: np.ndarray) -> float:
        return float(np.dot(self.w, x) + self.bias)
    async def update(self, x: np.ndarray, y_true: float, token_ids: List[int],
                     attention_bundle: Optional[Dict[str, Any]] = None) -> float:
        async with self._lock:
            y_hat = self.predict(x)
            self.last_output = y_hat
            error = y_true - y_hat
            self.last_error = error
            self.total_error_sq += error * error
            self.update_count += 1

            # Guard conditions from nlms.py - skip update if conditions met
            if self.guard_sign:
                # Skip if sign mismatch but error is small (prevents unnecessary updates)
                if (y_hat > 0) != (y_true > 0) and abs(error) < self.error_thresh:
                    return y_hat

            if self.arousal_band is not None:
                # Skip if target is in arousal band and error is small
                lo, hi = self.arousal_band
                if (lo <= y_true <= hi) and abs(error) < self.error_thresh:
                    return y_hat

            attention_gains = self.spiking_attention.compute_gains(token_ids, self.vocab_size)
            avg_gain = 1.0
            if attention_gains is not None and token_ids:
                gains_for_seq = [attention_gains[token] for token in token_ids if 0 <= token < self.vocab_size]
                if gains_for_seq: avg_gain = np.mean(gains_for_seq)

            multi_mu_scalar = 1.0
            if attention_bundle is not None and self.multi_attention is not None:
                bundle_tokens = attention_bundle.get('token_ids', token_ids)
                mc_result = self.multi_attention.compute(
                    bundle_tokens,
                    attention_bundle.get('amp'),
                    attention_bundle.get('pitch'),
                    attention_bundle.get('boundary'),
                )
                if mc_result:
                    multi_mu_scalar = mc_result.get('mu_scalar', 1.0)

            self.mu = max(self.mu_min, self.mu * self.mu_decay)
            modulated_mu = self.mu * avg_gain * multi_mu_scalar

            # Adaptive learning rate based on error magnitude (larger errors = faster learning)
            error_magnitude = abs(error)
            adaptive_factor = 1.0 + min(2.0, error_magnitude / 10.0)  # Boost LR for large errors
            modulated_mu *= adaptive_factor

            # Clip error to prevent extreme updates (but allow larger errors)
            clipped_error = np.clip(error, -100.0, 100.0)

            x_norm_sq = np.dot(x, x) + self.epsilon
            step = (modulated_mu * clipped_error * x) / x_norm_sq

            # Clip step size to prevent divergence (increased threshold)
            step_norm = np.linalg.norm(step)
            max_step_norm = 3.0  # Increased further to allow larger updates
            if step_norm > max_step_norm:
                step = step * (max_step_norm / step_norm)

            self.w += step

            # Update bias term with adaptive learning (more aggressive for large errors)
            bias_learning_rate = 0.3 * adaptive_factor  # More aggressive bias learning
            bias_step = modulated_mu * clipped_error * bias_learning_rate
            self.bias += bias_step

            # Clip weights to prevent unbounded growth
            weight_norm = np.linalg.norm(self.w)
            max_weight_norm = 100.0
            if weight_norm > max_weight_norm:
                self.w = self.w * (max_weight_norm / weight_norm)

            # Clip bias to reasonable range
            self.bias = np.clip(self.bias, -50.0, 50.0)

            return y_hat
    def get_rmse(self) -> float:
        if self.update_count == 0: return float('inf')
        return float(np.sqrt(self.total_error_sq / self.update_count))
    def state_dict(self) -> Dict:
        return {"w": self.w, "bias": self.bias, "mu": self.mu, "update_count": self.update_count}
    def load_state_dict(self, state: Dict):
        self.w = state["w"]
        self.bias = state.get("bias", 0.0)
        self.mu = state.get("mu", self.mu_initial)
        self.update_count = state.get("update_count", 0)

# --- 11. Liquid/MoE Components ---
# Bandit-based gating for expert selection
class BanditGating:
    """UCB-based bandit gating to track and select best-performing experts"""
    def __init__(self, n_experts: int, exploration_factor: float = 2.0):
        self.n_experts = n_experts
        self.exploration_factor = exploration_factor
        # Track rewards (negative errors) and counts for each expert
        self.total_rewards = np.zeros(n_experts, dtype=np.float64)
        self.selection_counts = np.ones(n_experts, dtype=np.float64)  # Start at 1 to avoid division by zero
        self.total_selections = n_experts  # Total times any expert was selected

    def update(self, expert_idx: int, error: float):
        """Update bandit statistics with expert performance"""
        # Convert error to reward (lower error = higher reward)
        reward = 1.0 / (1.0 + abs(error))  # Reward in [0, 1]
        self.total_rewards[expert_idx] += reward
        self.selection_counts[expert_idx] += 1.0
        self.total_selections += 1

    def get_ucb_scores(self) -> np.ndarray:
        """Compute UCB scores for all experts"""
        # Average reward
        avg_rewards = self.total_rewards / self.selection_counts

        # Confidence bound
        confidence = self.exploration_factor * np.sqrt(
            np.log(self.total_selections + 1) / self.selection_counts
        )

        # UCB = average reward + confidence bound
        ucb_scores = avg_rewards + confidence
        return ucb_scores

    def select_top_k(self, k: int, base_gates: np.ndarray) -> tuple:
        """Select top-k experts using UCB scores, modulated by base gates"""
        ucb_scores = self.get_ucb_scores()

        # Combine UCB scores with base gates (weighted combination)
        # UCB provides exploration/exploitation, gates provide input-specific routing
        combined_scores = 0.7 * base_gates + 0.3 * ucb_scores

        # Select top-k
        topk_idx = np.argsort(combined_scores)[-k:][::-1]

        # Normalize gates
        topk_gates = combined_scores[topk_idx]
        topk_gates = np.maximum(topk_gates, 0.0)  # Ensure non-negative
        gate_sum = np.sum(topk_gates)
        if gate_sum > 0:
            topk_gates = topk_gates / gate_sum
        else:
            topk_gates = np.ones(k) / k  # Uniform if all zero

        return topk_idx, topk_gates

    def reset(self):
        """Reset bandit statistics"""
        self.total_rewards.fill(0.0)
        self.selection_counts.fill(1.0)
        self.total_selections = self.n_experts

@dataclass
class LiquidCell:
    in_dim: int; hidden_dim: int; dt: float = 0.02
    tau_min: float = 0.02; tau_max: float = 2.0
    rng: np.random.Generator = field(default_factory=lambda: np.random.default_rng(1337))
    W: np.ndarray = field(init=False); U: np.ndarray = field(init=False)
    b: np.ndarray = field(init=False); V: np.ndarray = field(init=False)
    c: np.ndarray = field(init=False); h: np.ndarray = field(init=False)
    def __post_init__(self):
        self.W = self.rng.normal(0, np.sqrt(2.0/(self.hidden_dim+self.hidden_dim)), (self.hidden_dim, self.hidden_dim))
        self.U = self.rng.normal(0, np.sqrt(2.0/(self.in_dim+self.hidden_dim)), (self.hidden_dim, self.in_dim))
        self.b = np.zeros((self.hidden_dim,), dtype=np.float64)
        self.V = self.rng.normal(0, np.sqrt(2.0/(self.in_dim+self.hidden_dim)), (self.hidden_dim, self.in_dim))
        self.c = self.rng.normal(0, 0.1, (self.hidden_dim,))
        self.h = np.zeros((self.hidden_dim,), dtype=np.float64)
    def reset(self): self.h[:] = 0.0
    def step(self, x: np.ndarray, energy: Optional[EnergyMeter] = None) -> np.ndarray:
        x = np.asarray(x, dtype=np.float64).reshape(-1)
        vx = self.V @ x + self.c; tau = self.tau_min + softplus(vx)
        tau = np.minimum(tau, self.tau_max)
        Wh = self.W @ self.h; Ux = self.U @ x
        a = tanh(Wh + Ux + self.b); dh = - self.h / np.maximum(tau, 1e-6) + a
        self.h = self.h + self.dt * dh
        if energy is not None: energy.add_macs((self.hidden_dim*self.hidden_dim) + (self.hidden_dim*self.in_dim))
        return self.h.copy()
    def state_dict(self) -> Dict: return {"W": self.W, "U": self.U, "b": self.b, "V": self.V, "c": self.c, "h": self.h}
    def load_state_dict(self, state: Dict):
        self.W = state["W"]; self.U = state["U"]; self.b = state["b"]
        self.V = state["V"]; self.c = state["c"]; self.h = state["h"]
@dataclass
class LiquidGatingNetwork:
    in_dim: int; hidden_dim: int; n_experts: int
    top_k: int = 2; temperature: float = 1.0
    usage_smoothing: float = 0.99; bias_lr: float = 0.01
    usage_beta: float = 0.5
    cell: LiquidCell = field(init=False); Wg: np.ndarray = field(init=False)
    bg: np.ndarray = field(init=False); usage_ma: np.ndarray = field(init=False)
    rng: np.random.Generator = field(default_factory=lambda: np.random.default_rng(2025))
    energy: EnergyMeter = field(default_factory=EnergyMeter)
    def __post_init__(self):
        self.cell = LiquidCell(self.in_dim, self.hidden_dim, rng=self.rng)
        self.Wg = self.rng.normal(0, np.sqrt(2.0/(self.hidden_dim+self.n_experts)), (self.n_experts, self.hidden_dim))
        self.bg = np.zeros((self.n_experts,), dtype=np.float64)
        self.usage_ma = np.zeros((self.n_experts,), dtype=np.float64)
        self._lock = asyncio.Lock()
    async def forward(self, x: np.ndarray, attn_gain: float = 1.0) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        async with self._lock:
            h = self.cell.step(x, self.energy)
            logits = (self.Wg @ h) + self.bg
            logits = self._apply_usage_bias(logits)
            temp = max(0.2, self.temperature / max(1e-6, attn_gain))
            probs = softmax(logits, temp=temp)
            k = max(1, min(self.top_k, self.n_experts))
            topk_idx = np.argpartition(probs, -k)[-k:]
            topk_probs = probs[topk_idx]
            if topk_probs.sum() <= 1e-12: gates = np.ones_like(topk_probs) / len(topk_probs)
            else: gates = topk_probs / topk_probs.sum()
            out = np.zeros_like(probs); out[topk_idx] = gates
            eps = 0.01
            if self.n_experts > 0 and eps > 0:
                j = int(np.argmin(self.usage_ma)); out = (1.0 - eps) * out; out[j] += eps
            self.usage_ma = self.usage_smoothing * self.usage_ma + (1.0 - self.usage_smoothing) * out
            # Track last winners for dopamine reward (from liquidmoe.py)
            self.last_winners = topk_idx
            return out, topk_idx, probs
    def _apply_usage_bias(self, logits: np.ndarray) -> np.ndarray:
        eps = 1e-6; target = 1.0 / self.n_experts
        inv_usage = target / (self.usage_ma + eps)
        return logits + self.usage_beta * np.log(inv_usage)
    async def apply_endocrine(self, *, cortisol: float = 0.0, gh: float = 0.0,
                             thyroid: float = 1.0, dopamine: float = 0.0, eps: Optional[float] = None) -> None:
        """Apply endocrine modulation to gating network (from liquidmoe.py)"""
        async with self._lock:
            # Temperature ↑ with cortisol (stress) — clamp for stability
            self.temperature = float(np.clip(self.temperature * (1.0 + 0.30 * cortisol), 0.5, 2.5))

            # Bias LR scales with thyroid (metabolic rate around 1.0 baseline)
            self.bias_lr = float(np.clip(self.bias_lr * (1.0 + 0.40 * (thyroid - 1.0)), 1e-4, 0.1))

            # Top-K capacity expands with GH (growth hormone), but never beyond n_experts
            base_top_k = getattr(self, 'base_top_k', self.top_k)
            self.base_top_k = base_top_k
            self.top_k = int(np.clip(round(base_top_k * (1.0 + 0.20 * gh)), 1, self.n_experts))

            # Dopamine: nudge most recent winners' biases (reward)
            if dopamine > 0 and hasattr(self, 'last_winners') and self.last_winners is not None:
                self.bg[self.last_winners] += 0.10 * float(dopamine)

            # Optional: exploration epsilon override
            if eps is not None:
                self.eps = float(np.clip(eps, 0.0, 0.05))
    async def nudge_for_load_balance(self) -> None:
        async with self._lock:
            if self.n_experts <= 0: return
            target = 1.0 / float(self.n_experts); delta = target - self.usage_ma
            self.bg += self.bias_lr * delta
    def reset(self): self.cell.reset(); self.usage_ma[:] = 0.0; self.energy.reset()
    def state_dict(self) -> Dict:
        return {"cell": self.cell.state_dict(), "Wg": self.Wg, "bg": self.bg, "usage_ma": self.usage_ma}
    def load_state_dict(self, state: Dict):
        self.cell.load_state_dict(state["cell"]); self.Wg = state["Wg"]; self.bg = state["bg"]; self.usage_ma = state["usage_ma"]
class NLMSExpertAdapter:
    def __init__(self, neuron: ExpertNLMSHead): self.neuron = neuron
    def predict(self, x: np.ndarray) -> float: return self.neuron.predict(x)
    async def update(self, x: np.ndarray, y_true: float, token_ids: List[int],
                     attention_bundle: Optional[Dict[str, Any]] = None) -> float:
        return await self.neuron.update(x, y_true, token_ids, attention_bundle)
    def state_dict(self) -> Dict: return self.neuron.state_dict()
    def load_state_dict(self, state: Dict): self.neuron.load_state_dict(state)
@dataclass
class LiquidMoERouter:
    experts: Dict[str, NLMSExpertAdapter]
    in_dim: int; hidden_dim: int; top_k: int = 2
    temperature: float = 1.0
    gating: LiquidGatingNetwork = field(init=False)
    bandit: BanditGating = field(init=False)
    names: List[str] = field(init=False)
    energy: EnergyMeter = field(default_factory=EnergyMeter)
    use_bandit: bool = True  # Enable bandit gating
    def __post_init__(self):
        self.names = list(self.experts.keys())
        self.gating = LiquidGatingNetwork(
            in_dim=self.in_dim, hidden_dim=self.hidden_dim,
            n_experts=len(self.names), top_k=self.top_k,
            temperature=self.temperature,
        )
        self.bandit = BanditGating(n_experts=len(self.names), exploration_factor=2.0)
    async def route(self, x: np.ndarray, attn_gain: float = 1.0) -> Dict[str, any]:
        gates_sparse, topk_idx_base, probs = await self.gating.forward(x, attn_gain=attn_gain)

        # Use bandit gating if enabled
        if self.use_bandit:
            # Convert sparse gates to dense for bandit
            gates_dense = np.zeros(len(self.names), dtype=np.float64)
            for i, idx in enumerate(topk_idx_base):
                gates_dense[int(idx)] = float(gates_sparse[i])

            # Get bandit-selected experts
            topk_idx, topk_gates = self.bandit.select_top_k(self.top_k, gates_dense)

            # Update gates_sparse and topk_idx
            gates_sparse = topk_gates
            topk_idx = topk_idx
        else:
            topk_idx = topk_idx_base

        chosen = [(self.names[i], float(gates_sparse[j])) for j, i in enumerate(topk_idx)]
        y = 0.0; per_expert: Dict[str, Dict[str, float]] = {}
        for j, i in enumerate(topk_idx):
            name = self.names[int(i)]; gate = float(gates_sparse[j])
            pred = float(self.experts[name].predict(x)); y += gate * pred
            self.energy.add_macs(self.in_dim); per_expert[name] = {"gate": gate, "pred": pred}
        return {'y_hat': float(y), 'topk': chosen, 'probs': probs, 'per_expert': per_expert,
            'energy_j': self.energy.total_j + self.gating.energy.total_j}
    async def learn(self, x: np.ndarray, token_ids: List[int], y_true: float,
                    attn_gain: float = 1.0, attention_bundle: Optional[Dict[str, Any]] = None) -> Dict[str, any]:
        out = await self.route(x, attn_gain=attn_gain); tasks = []
        expert_errors = {}  # Track errors for bandit updates

        for name, info in out['per_expert'].items():
            gate = float(info['gate']);
            if gate <= 0.0: continue
            target = float(y_true)
            pred = float(info['pred'])
            error = abs(target - pred)
            expert_errors[name] = error
            tasks.append(self.experts[name].update(x, target, token_ids, attention_bundle))

        await asyncio.gather(*tasks)

        # Update bandit with expert performance
        if self.use_bandit:
            for name, error in expert_errors.items():
                if name in self.names:
                    expert_idx = self.names.index(name)
                    self.bandit.update(expert_idx, error)

        await self.gating.nudge_for_load_balance(); return out
    def reset(self):
        self.gating.reset()
        self.energy.reset()
        if self.use_bandit:
            self.bandit.reset()
    def state_dict(self) -> Dict:
        expert_states = {name: expert.state_dict() for name, expert in self.experts.items()}
        bandit_state = {
            "total_rewards": self.bandit.total_rewards.tolist(),
            "selection_counts": self.bandit.selection_counts.tolist(),
            "total_selections": self.bandit.total_selections
        } if self.use_bandit else None
        return {"gating": self.gating.state_dict(), "experts": expert_states, "bandit": bandit_state}
    def load_state_dict(self, state: Dict):
        self.gating.load_state_dict(state["gating"])
        for name, expert_state in state["experts"].items():
            if name in self.experts: self.experts[name].load_state_dict(expert_state)
        if self.use_bandit and "bandit" in state and state["bandit"]:
            bandit_state = state["bandit"]
            self.bandit.total_rewards = np.array(bandit_state["total_rewards"])
            self.bandit.selection_counts = np.array(bandit_state["selection_counts"])
            self.bandit.total_selections = bandit_state["total_selections"]

# --- 12. "Temporal Mind" ---
class TemporalMemoryInterpolator:
    def __init__(self, epsilon: float = 1e-12):
        self.epsilon = epsilon
        print("CREATED: TemporalMemoryInterpolator (Aura 7.0 Hippocampus)")
    def interpolate(self, M0: np.ndarray, M1: np.ndarray, t: float,
                    mode: Literal['linear', 'fourier', 'hilbert', 'hamiltonian'] = 'hilbert'
                   ) -> np.ndarray:
        alpha = np.clip(t, 0.0, 1.0)
        if mode == 'linear': return (1.0 - alpha) * M0 + alpha * M1
        elif mode == 'fourier':
            F0 = np.fft.fft(M0); F1 = np.fft.fft(M1)
            F_interp = (1.0 - alpha) * F0 + alpha * F1
            return np.real(np.fft.ifft(F_interp))
        A0 = hilbert(M0, axis=0); A1 = hilbert(M1, axis=0)
        if mode == 'hilbert':
            A_interp = (1.0 - alpha) * A0 + alpha * A1
            return np.real(A_interp)
        elif mode == 'hamiltonian':
            print("WARNING: Hamiltonian mode is computationally expensive.")
            A_diff = (A1 - A0).astype(np.complex128)
            H_num = np.outer(A_diff, A_diff.T.conj())
            H_den = np.linalg.norm(A_diff)**2 + self.epsilon
            H = H_num / H_den; U = expm(-1j * H * alpha)
            A_interp = U @ A0; return np.real(A_interp)
        else: raise ValueError(f"Unknown interpolation mode: {mode}")

# --- 13. CNS & ThalamicRouter (Trained) ---
class ConsciousnessLevel(Enum):
    DEEP_SLEEP = 0; ASLEEP = 1; ALERT = 2; FOCUSED = 3; HYPERVIGILANT = 4
class CentralNervousSystem:
    def __init__(self):
        self.consciousness_level = ConsciousnessLevel.ALERT
        self.stress_level = 0.0 # 'cortisol'
        self.consolidation_factor = 1.0 # Reduces impact of errors after dreaming
        print("CREATED: CentralNervousSystem (CNS)")
    def set_consciousness(self, level: ConsciousnessLevel):
        if self.consciousness_level != level:
            self.consciousness_level = level; print(f"CNS: Consciousness set to {level.name}")
    def update_stress(self, error: float):
        new_stress = abs(error) * 1.5 * self.consolidation_factor
        self.stress_level = (self.stress_level * 0.5) + (new_stress * 0.5)
        self.stress_level = max(0.0, self.stress_level - 0.1)
        if self.stress_level > 1.0: self.set_consciousness(ConsciousnessLevel.HYPERVIGILANT)
        else: self.set_consciousness(ConsciousnessLevel.ALERT)
    def apply_consolidation(self, factor: float = 0.7):
        """Apply memory consolidation effect that reduces impact of future errors"""
        self.consolidation_factor = factor

    def get_endocrine_levels(self) -> Dict[str, float]:
        """Compute endocrine hormone levels from CNS state (for liquidmoe.py integration)"""
        # Cortisol: stress level (0-1 maps to 0-2)
        cortisol = min(2.0, self.stress_level * 2.0)

        # Growth Hormone (GH): higher when learning (low stress, alert)
        gh = 0.5 if self.consciousness_level == ConsciousnessLevel.ALERT else 0.0

        # Thyroid: metabolic rate (higher when alert, lower when stressed)
        thyroid = 1.0 - (self.stress_level * 0.3)  # 1.0 baseline, down to 0.7 when stressed

        # Dopamine: reward signal (higher when stress is low and consciousness is alert)
        dopamine = max(0.0, 1.0 - self.stress_level) if self.consciousness_level == ConsciousnessLevel.ALERT else 0.0

        return {
            'cortisol': cortisol,
            'gh': gh,
            'thyroid': thyroid,
            'dopamine': dopamine
        }
class TrainedThalamicRouter:
    """The 'School' - runs the offline-trained PyTorch model"""
    def __init__(self, model_path: str, label_maps: Dict, feature_gen: FeatureGenerator):
        self.label_maps = label_maps
        self.emotion_map = label_maps.get('emotion', {})
        self.intent_map = label_maps.get('intent', {})
        self.emotion_labels_inv = {v: k for k, v in self.emotion_map.items()}
        self.feature_gen = feature_gen; self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        try:
            self.model = MultiTaskHead(
                feature_gen.TOTAL_FEATURES,
                len(self.emotion_map), len(self.intent_map), len(self.emotion_map) # Mock tone
            ).to(self.device)
            self.model.load_state_dict(torch.load(model_path))
            self.model.eval()
            print(f"CREATED: TrainedThalamicRouter (Loaded from {model_path})")
        except Exception as e:
            print(f"--- ❌ ERROR: Failed to load ThalamicRouter model. {e} ---")

    def get_target_signals(self, query: str, verbose: bool = True) -> dict:
        if not self.model: return {'emotion': 0.0, 'intent': 0.0}
        record = {"text": query, "plutchik": {"primary": "neutral"}}
        # --- THE FIX: We must use use_emotion_sine=False ---
        sbert_vec = self.feature_gen.sbert_model.encode(query, normalize_embeddings=True)
        x_raw = self.feature_gen.build_features(record, sbert_vec, use_emotion_sine=False)
        x_tensor = torch.tensor(x_raw, dtype=torch.float32).unsqueeze(0).to(self.device)
        with torch.no_grad():
            emo_logits, intent_logits, _ = self.model(x_tensor)
            emo_pred_index = emo_logits.argmax(dim=1).item()
            intent_pred_index = intent_logits.argmax(dim=1).item()
        pred_label = self.emotion_labels_inv.get(emo_pred_index, 'neutral')
        signals = {'emotion': float(emo_pred_index), 'intent': float(intent_pred_index)}
        if verbose:
            print(f"Thalamus Predicted: '{pred_label}' (Index: {emo_pred_index})")
        return signals

# --- 14. The Final IBNN (Aura 7.0 - Hebbian-MoE-Temporal) ---
class IntegratedBioNeuralNetwork:
    """
    Aura 7.0: Integrates all components into a persistent,
    unsupervised-learning, and time-aware architecture.
    """
    def __init__(self, n_experts: int = 15, hebbian_components: int = 64,
                 llm_model_name: str = "meta-llama/Llama-3.2-3B-Instruct"):
        print("--- 🧠 Initializing Aura 7.0 (Hebbian-MoE-Temporal) ---")
        if torch.cuda.is_available():
            print(f"   -> GPU: {torch.cuda.get_device_name(0)}, VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
            maybe_empty_cuda_cache("startup", min_free_ratio=0.05)
        self.feature_gen = FeatureGenerator()
        self.response_gen = ResponseGenerator(llm_model_name)
        self.cns = CentralNervousSystem()
        self.interpolator = TemporalMemoryInterpolator()
        self.raw_feature_dim = self.feature_gen.TOTAL_FEATURES
        self.vocab_size = self.feature_gen.vocab_size
        self.abstract_feature_dim = hebbian_components

        self.whitener: OptimizedWhitener = None
        self.hippocampus: OjaLayer = None # Hebbian Cortex
        self.cortex: LiquidMoERouter = None # MoE Cortex
        self.router: Optional[TrainedThalamicRouter] = None
        self.n_experts = n_experts

        self.education_dir = SAVE_DIR # Use global
        os.makedirs(self.education_dir, exist_ok=True)
        print("--- Aura 7.0 Brain Initialized (Awaiting Education) ---")

    def _create_brain_layers(self, hebbian_k: int):
        """Dynamically creates the brain layers based on the Hebbian cortex size."""
        self.abstract_feature_dim = hebbian_k
        print(f"--- 🧠 Building Dynamic Brain Layers ---")
        print(f"   -> Hebbian Cortex (Oja) K = {hebbian_k}")

        self.hippocampus = OjaLayer(
            dim=self.raw_feature_dim,
            n_components=hebbian_k,
            mode='nonlinear', lr=5e-4, max_components=128, grow_threshold=0.3
        )
        self.whitener = OptimizedWhitener(dim=self.raw_feature_dim)

        experts: Dict[str, NLMSExpertAdapter] = {}
        attention_config = {
            'decay': 0.7,
            'theta': 1.0,
            'k_winners': 3,
            'gain_up': 1.5,
            'gain_down': 0.7,
            'multi_channel': {
                'k_winners': 5,
                'w_amp': 1.0,
                'w_pitch': 1.4,
                'w_bound': 0.8,
                'gain_up': 2.0,
                'gain_down': 0.5,
                'use_stdp': True,
                'smoothing': 1,
            }
        }
        num_emotions = len(GOEMOTIONS_LABELS)
        emotion_range_per_expert = num_emotions / self.n_experts

        for i in range(self.n_experts):
            name = f"expert__{i}"
            # Initialize bias toward expert's emotion range (0-27 for GoEmotions)
            target_emotion_idx = i * emotion_range_per_expert + emotion_range_per_expert / 2
            head = ExpertNLMSHead(
                n_features=self.abstract_feature_dim,
                vocab_size=self.vocab_size,
                attention_config=attention_config,
                mu=0.5,  # Even higher learning rate for faster convergence
                mu_decay=0.99995,  # Very slow decay to maintain learning longer
                mu_min=0.1,  # Higher minimum learning rate
                initial_bias=target_emotion_idx  # Initialize bias to target range
            )
            # Initialize weights with larger random values for better coverage
            head.w = np.random.normal(0, 1.0, self.abstract_feature_dim)
            # Ensure bias is actually set (double-check)
            head.bias = float(target_emotion_idx)
            experts[name] = NLMSExpertAdapter(neuron=head)

        self.cortex = LiquidMoERouter(
            experts=experts, in_dim=self.abstract_feature_dim, # Use K
            hidden_dim=128, top_k=3
        )
        print(f"--- 🧠 Dynamic Brain Layers Created ---")

    async def educate_brain(self):
        """
        Runs the 'School' pipeline to pre-train all components.

        """
        print("\n--- 🎓 STARTING OFFLINE EDUCATION (GoEmotions) ---")
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # 1. Load GoEmotions
        print("Loading 'google-research-datasets/go_emotions'...")
        dataset = load_dataset("google-research-datasets/go_emotions", split='train')
        dataset = dataset.shuffle(seed=42).select(range(30000))
        all_texts = dataset['text']

        # 2. Precompute SBERT
        print(f"Pre-computing SBERT embeddings for {len(all_texts)} texts...")
        sbert_embeddings = await asyncio.to_thread(
            self.feature_gen.sbert_model.encode,
            all_texts, normalize_embeddings=True, batch_size=256
        )

        # 3. Create Label Maps
        emotion_labels = GOEMOTIONS_LABELS
        intent_labels = ["question", "statement", "exclamation", "request", "none"]
        tone_labels = emotion_labels # Mock
        label_maps = {
            'emotion': {label: idx for idx, label in enumerate(emotion_labels)},
            'intent': {label: idx for idx, label in enumerate(intent_labels)},
            'tone': {label: idx for idx, label in enumerate(tone_labels)},
        }

        # 4. Build Features
        print("Building features...")
        X_features, y_emotion, y_intent, y_tone = [], [], [], []
        X_features_hebbian_raw = [] # For Oja training

        for i, record in enumerate(dataset):
            primary_emotion = 'neutral'
            label_indices = record['labels']
            if label_indices:
                for idx in label_indices:
                    if 0 <= idx < len(GOEMOTIONS_LABELS):
                        label_name = GOEMOTIONS_LABELS[idx]
                        if label_name != 'neutral':
                            primary_emotion = label_name; break

            mock_record = {
                "text": record['text'],
                "plutchik": {"primary": primary_emotion, "intensity": 1.0},
                "intent": "question" if "?" in record['text'] else "statement",
                "tone": primary_emotion
            }

            # --- THE FIX ---
            # Features for Thalamus (teacher) -> NO emotional sine wave (no cheating)
            X_features.append(self.feature_gen.build_features(mock_record, sbert_embeddings[i], use_emotion_sine=False))
            # Features for Hebbian (unsupervised) -> YES emotional sine wave (part of context)
            X_features_hebbian_raw.append(self.feature_gen.build_features(mock_record, sbert_embeddings[i], use_emotion_sine=True))
            # --- END FIX ---

            y_emotion.append(label_maps['emotion'].get(primary_emotion, GOEMOTIONS_MAP['neutral']))
            y_intent.append(label_maps['intent'].get(mock_record['intent'], 1))
            y_tone.append(label_maps['tone'].get(primary_emotion, GOEMOTIONS_MAP['neutral']))

        X_train_np = np.stack(X_features)
        y_emotion_np = np.array(y_emotion)

        indices = np.arange(len(X_train_np))
        X_train_split, X_test_split, y_emotion_train_split, y_emotion_test_split, \
        y_intent_train_split, y_intent_test_split, y_tone_train_split, y_tone_test_split = \
            train_test_split(X_train_np, y_emotion_np, y_intent, y_tone, test_size=0.2, random_state=42, stratify=y_emotion_np)

        X_train_torch = torch.tensor(X_train_split, dtype=torch.float32).to(device)
        y_emotion_train_torch = torch.tensor(y_emotion_train_split, dtype=torch.long).to(device)
        y_intent_train_torch = torch.tensor(y_intent_train_split, dtype=torch.long).to(device)
        y_tone_train_torch = torch.tensor(y_tone_train_split, dtype=torch.long).to(device)

        X_test_torch = torch.tensor(X_test_split, dtype=torch.float32).to(device)
        y_emotion_test_torch = torch.tensor(y_emotion_test_split, dtype=torch.long).to(device)
        y_intent_test_torch = torch.tensor(y_intent_test_split, dtype=torch.long).to(device)
        y_tone_test_torch = torch.tensor(y_tone_test_split, dtype=torch.long).to(device)

        # --- 5. Train the "Hebbian Cortex" (OjaLayer) ---
        print("Training Hebbian Cortex (OjaLayer) on whitened data...")
        self._create_brain_layers(hebbian_k=self.abstract_feature_dim)

        # Whiten the *full-context* features for Hebbian learning
        X_train_np_hebbian = np.stack(X_features_hebbian_raw)
        X_features_whitened = np.array([self.feature_gen.whitener.transform(x) for x in X_train_np_hebbian])

        for x_w in X_features_whitened:
             self.hippocampus.step(x_w)

        final_k = self.hippocampus.K
        if final_k != self.abstract_feature_dim:
            print(f"Hebbian Cortex grew from {self.abstract_feature_dim} to {final_k}. Rebuilding MoE...")
            self._create_brain_layers(hebbian_k=final_k)

        print(f"   -> Hebbian Cortex (OjaLayer) training complete. Final K={final_k}")

        # --- 6. Train the "Thalamus" (MultiTaskHead) ---
        print(f"Training MultiTask ThalamicRouter...")
        class_weights_emotion = compute_class_weight('balanced', classes=np.unique(y_emotion_train_split), y=y_emotion_train_split)
        class_weights_emotion = torch.tensor(class_weights_emotion, dtype=torch.float32).to(device)

        thalamus_model = MultiTaskHead(
            self.feature_gen.TOTAL_FEATURES,
            len(label_maps['emotion']), len(label_maps['intent']), len(label_maps['tone'])
        ).to(device)

        batch_size = 1024
        num_epochs = 500
        initial_lr = 3e-4

        optimizer = optim.AdamW(thalamus_model.parameters(), lr=initial_lr, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

        criterion_emotion = nn.CrossEntropyLoss(weight=class_weights_emotion)
        criterion_intent = nn.CrossEntropyLoss()
        criterion_tone = nn.CrossEntropyLoss(weight=class_weights_emotion)

        train_dataset = torch.utils.data.TensorDataset(
            X_train_torch, y_emotion_train_torch, y_intent_train_torch, y_tone_train_torch
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        best_val_loss = float('inf')
        epochs_no_improve = 0
        patience = 8

        for epoch in range(num_epochs):
            thalamus_model.train()
            epoch_train_loss = 0.0
            num_batches = 0

            for batch_x, batch_emo, batch_intent, batch_tone in train_loader:
                optimizer.zero_grad()
                emo_out, intent_out, tone_out = thalamus_model(batch_x)
                loss = (1.0 * criterion_emotion(emo_out, batch_emo) +
                        0.5 * criterion_intent(intent_out, batch_intent) +
                        0.5 * criterion_tone(tone_out, batch_tone))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(thalamus_model.parameters(), max_norm=1.0)
                optimizer.step()
                epoch_train_loss += loss.item()
                num_batches += 1

            avg_train_loss = epoch_train_loss / num_batches

            thalamus_model.eval()
            with torch.no_grad():
                emo_val, intent_val, tone_val = thalamus_model(X_test_torch)
                val_loss = (1.0 * criterion_emotion(emo_val, y_emotion_test_torch) +
                            0.5 * criterion_intent(intent_val, y_intent_test_torch) +
                            0.5 * criterion_tone(tone_val, y_tone_test_torch))

            scheduler.step(val_loss)
            current_lr = optimizer.param_groups[0]['lr']

            print(f"Epoch {epoch+1:02d}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss.item():.4f} | LR: {current_lr:.6f}")

            # Clear GPU cache only if VRAM gets tight
            if (epoch + 1) % 10 == 0:
                maybe_empty_cuda_cache(f"thalamus training epoch {epoch+1}", min_free_ratio=0.08)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                thalamus_path = os.path.join(self.education_dir, "thalamic_router_multitask.pt")
                torch.save(thalamus_model.state_dict(), thalamus_path)
                print(f"   -> New best model saved to {thalamus_path}")
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"   -> Early stopping on epoch {epoch+1}.")
                    break

        # --- 7. "Mind Upload" - Initialize Router ---
        self.router = TrainedThalamicRouter(thalamus_path, label_maps, self.feature_gen)

        # --- 8. Warm-up MoE Experts ---
        print("Warming up MoE experts on training data...")
        warmup_sweeps = 3
        warmup_samples = min(2000, len(dataset))
        warmup_val_samples = min(500, len(dataset))
        warmup_val_indices = np.random.choice(len(dataset), warmup_val_samples, replace=False)

        for sweep in range(warmup_sweeps):
            print(f"   --> Warm-up sweep {sweep+1}/{warmup_sweeps}")
            warmup_indices = np.random.choice(len(dataset), warmup_samples, replace=False)
            errors = []
            predictions = []
            targets = []
            for i, idx in enumerate(warmup_indices):
                record = dataset[int(idx)]
                query_text = record['text']

                target_signals = self.router.get_target_signals(query_text, verbose=False)
                y_true_emotion_idx = target_signals['emotion']
                y_true_emotion_label = self.router.emotion_labels_inv.get(y_true_emotion_idx, 'neutral')

                x_raw, token_ids, attention_bundle = self.feature_gen.generate_for_query(query_text, thalamus_prediction=y_true_emotion_label)
                xw = self.feature_gen.whitener.transform(x_raw)
                oja_out = self.hippocampus.step(xw)
                y_abstract = oja_out.y

                target_for_moe = float(y_true_emotion_idx)
                out = await self.cortex.learn(x=y_abstract, token_ids=token_ids, y_true=target_for_moe, attention_bundle=attention_bundle)

                pred = out['y_hat']
                _, discrete_pred = quantize_moe_prediction(pred, NUM_GOEMOTIONS)
                error = abs(target_for_moe - discrete_pred)
                errors.append(error)
                predictions.append(discrete_pred)
                targets.append(target_for_moe)

                if (i + 1) % 200 == 0:
                    avg_error = np.mean(errors[-200:])
                    avg_pred = np.mean(predictions[-200:]) if predictions else 0.0
                    avg_target = np.mean(targets[-200:]) if targets else 0.0
                    print(f"      Sweep {sweep+1} Progress {i+1}/{warmup_samples}: Error={avg_error:.2f}, Pred={avg_pred:.2f}, Target={avg_target:.2f}")

            sweep_mae = np.mean(errors) if errors else float('inf')
            sweep_rmse = np.sqrt(np.mean([e*e for e in errors])) if errors else float('inf')
            sweep_corr = np.corrcoef(predictions, targets)[0, 1] if len(predictions) > 1 else 0.0
            print(f"   -> Sweep {sweep+1} metrics: MAE={sweep_mae:.2f}, RMSE={sweep_rmse:.2f}, Corr={sweep_corr:.3f}")

            await self._evaluate_moe_subset(
                dataset,
                warmup_val_indices,
                description=f"Validation after sweep {sweep+1}"
            )

        print(f"   -> MoE experts warmed up with {warmup_sweeps} sweeps ({warmup_samples} samples each)")
        await self._evaluate_moe_subset(
            dataset,
            warmup_val_indices,
            description="Final warm-up validation"
        )

        print("--- 🎓 OFFLINE EDUCATION COMPLETE ---")

    async def _evaluate_moe_subset(self, dataset, indices, description: str = "Validation"):
        if indices is None or len(indices) == 0:
            return
        if not self.router or not self.cortex:
            print(f"[WARN] Cannot run {description}; router or cortex missing.")
            return
        errors = []
        predictions = []
        targets = []
        for idx in indices:
            record = dataset[int(idx)]
            query_text = record.get('text', '')
            if not query_text:
                continue
            target_signals = self.router.get_target_signals(query_text, verbose=False)
            y_true_emotion_idx = target_signals['emotion']
            y_true_emotion_label = self.router.emotion_labels_inv.get(y_true_emotion_idx, 'neutral')

            x_raw, token_ids, attention_bundle = self.feature_gen.generate_for_query(query_text, thalamus_prediction=y_true_emotion_label)
            xw = self.feature_gen.whitener.transform(x_raw)
            oja_out = self.hippocampus.step(xw)
            y_abstract = oja_out.y

            out = await self.cortex.route(x=y_abstract, attn_gain=1.0)
            pred = out['y_hat']
            _, discrete_pred = quantize_moe_prediction(pred, NUM_GOEMOTIONS)
            target_val = float(y_true_emotion_idx)
            error = abs(target_val - discrete_pred)
            errors.append(error)
            predictions.append(discrete_pred)
            targets.append(target_val)
        if not errors:
            return
        mae = np.mean(errors)
        rmse = np.sqrt(np.mean([e*e for e in errors]))
        corr = np.corrcoef(predictions, targets)[0, 1] if len(predictions) > 1 else 0.0
        print(f"   -> {description}: MAE={mae:.2f}, RMSE={rmse:.2f}, Corr={corr:.3f}")

    async def process_query(self, query: str, update_stress: bool = True) -> str:
        print(f"\n--- ☀️ PROCESSING QUERY (Aura 7.0): '{query}' ---")
        if not self.router: return "I am uneducated. Please run `await aura.educate_brain()`."

        # 1. "Thalamus" (Educated) predicts the target emotion
        target_signals = self.router.get_target_signals(query)
        y_true_emotion_idx = target_signals['emotion']
        y_true_emotion_label = self.router.emotion_labels_inv.get(y_true_emotion_idx, 'neutral')

        # 2. "Ears" -> "Hebbian Cortex"
        # The Hebbian layer's feature vector *includes* the "correct" emotional sine wave
        # This is now part of the "context" for the MoE
        x_raw, token_ids, attention_bundle = self.feature_gen.generate_for_query(query, thalamus_prediction=y_true_emotion_label)
        xw = self.feature_gen.whitener.transform(x_raw)
        oja_out = self.hippocampus.step(xw)
        y_abstract = oja_out.y

        # 3. "MoE Cortex" (Online Learning)
        # The MoE experts learn to map the abstract engram to the Thalamus's target index
        target_for_moe = float(y_true_emotion_idx)

        out = await self.cortex.learn(
            x=y_abstract, token_ids=token_ids, y_true=target_for_moe, attention_bundle=attention_bundle
        )

        # 4. "CNS" (Feeling)
        raw_prediction = out['y_hat']
        moe_idx, discrete_prediction = quantize_moe_prediction(raw_prediction, NUM_GOEMOTIONS)
        prediction_error = target_for_moe - discrete_prediction
        if update_stress:
            self.cns.update_stress(prediction_error)
            # Apply endocrine modulation to gating network (from liquidmoe.py)
            endocrine_levels = self.cns.get_endocrine_levels()
            await self.cortex.gating.apply_endocrine(**endocrine_levels)

        print(f"Hebbian (Oja) Residual: {oja_out.residual_ema:.3f} (Grew: {oja_out.grew})")
        print(f"MoE Prediction: idx={moe_idx} (raw={raw_prediction:.2f}), Target: {int(target_for_moe)}, Error: {abs(prediction_error):.2f}")

        # Diagnostic: Show top expert contributions
        if 'topk' in out and len(out['topk']) > 0:
            top_expert = out['topk'][0]
            expert_name, expert_gate = top_expert
            expert_neuron = self.cortex.experts[expert_name].neuron
            expert_rmse = expert_neuron.get_rmse()
            expert_lr = expert_neuron.mu
            print(f"   -> Top Expert: {expert_name} (gate={expert_gate:.3f}, RMSE={expert_rmse:.2f}, LR={expert_lr:.4f})")

        # 5. "Mouth" speaks
        final_brain_state = {
            'cns_state': self.cns.consciousness_level,
            'stress_level': self.cns.stress_level,
        }
        response = await self.response_gen.generate_response(query, final_brain_state)
        self.last_run_final_stress = self.cns.stress_level
        return response

    def get_moe_statistics(self) -> Dict[str, Any]:
        """Get diagnostic statistics about MoE experts"""
        stats = {
            'expert_count': len(self.cortex.experts),
            'experts': {}
        }
        for name, expert_adapter in self.cortex.experts.items():
            neuron = expert_adapter.neuron
            stats['experts'][name] = {
                'rmse': neuron.get_rmse(),
                'learning_rate': neuron.mu,
                'update_count': neuron.update_count,
                'last_error': neuron.last_error
            }
        return stats

    def _get_brain_state_vector(self) -> np.ndarray:
        all_weights = []
        all_weights.append(self.feature_gen.whitener.mu.flatten())
        all_weights.append(self.feature_gen.whitener.var.flatten())
        all_weights.append(self.hippocampus.W.flatten())
        all_weights.append(self.cortex.gating.cell.W.flatten())
        all_weights.append(self.cortex.gating.cell.U.flatten())
        all_weights.append(self.cortex.gating.cell.b.flatten())
        all_weights.append(self.cortex.gating.cell.V.flatten())
        all_weights.append(self.cortex.gating.cell.c.flatten())
        all_weights.append(self.cortex.gating.Wg.flatten())
        all_weights.append(self.cortex.gating.bg.flatten())
        for expert in self.cortex.experts.values():
            all_weights.append(expert.neuron.w.flatten())
        return np.concatenate(all_weights)

    def _load_brain_state_vector(self, M: np.ndarray):
        print("--- 🧠 Loading brain state from vector... ---")
        idx = 0
        def _load(target_array: np.ndarray) -> int:
            nonlocal idx
            n = target_array.size
            try:
                target_array[:] = M[idx : idx + n].reshape(target_array.shape)
            except ValueError:
                print(f"!! Shape mismatch during load. Target: {target_array.shape}, Got: {M[idx : idx + n].shape}")
                min_n = min(target_array.size, M[idx:].size)
                if min_n == 0: return idx
                target_flat = target_array.flat; source_flat = M[idx:].flat
                for i in range(min_n): target_flat[i] = source_flat[i]
                n = min_n
            return idx + n
        try:
            idx = _load(self.feature_gen.whitener.mu)
            idx = _load(self.feature_gen.whitener.var)
            idx = _load(self.hippocampus.W)
            idx = _load(self.cortex.gating.cell.W)
            idx = _load(self.cortex.gating.cell.U)
            idx = _load(self.cortex.gating.cell.b)
            idx = _load(self.cortex.gating.cell.V)
            idx = _load(self.cortex.gating.cell.c)
            idx = _load(self.cortex.gating.Wg)
            idx = _load(self.cortex.gating.bg)
            for expert in self.cortex.experts.values():
                idx = _load(expert.neuron.w)
            print(f"--- 🧠 Brain state loaded successfully. Total params: {idx} ---")
        except Exception as e:
            print(f"--- ❌ ERROR: Failed to load brain state vector. {e} ---")

    async def save_keyframe(self, name: str):
        path = os.path.join(self.education_dir, f"keyframe_{name}_{int(time.time())}.npy")
        print(f"\n--- 💾 Saving keyframe to {path} ---")
        M = self._get_brain_state_vector()
        await asyncio.to_thread(np.save, path, M)
        print("--- 💾 Keyframe save complete ---")

    async def run_sleep_cycle(self, keyframe1_name: str, keyframe2_name: str):
        """
        Implements "Dream Recall" from the paper.
        Consolidates memories and reduces stress through dream interpolation.
        """
        print(f"\n--- 💤 RUNNING SLEEP CYCLE (Dreaming) ---")
        print(f"Loading keyframes: {keyframe1_name} and {keyframe2_name}")
        try:
            M0_path = sorted(glob.glob(os.path.join(self.education_dir, f"keyframe_{keyframe1_name}*.npy")))[-1]
            M1_path = sorted(glob.glob(os.path.join(self.education_dir, f"keyframe_{keyframe2_name}*.npy")))[-1]
            M0 = np.load(M0_path)
            M1 = np.load(M1_path)
        except Exception as e:
            print(f"--- ❌ ERROR: Could not load keyframes for dreaming. {e} ---"); return

        current_brain_shape = self._get_brain_state_vector().shape
        if M0.shape != M1.shape or M0.shape != current_brain_shape:
            print(f"--- ❌ ERROR: Keyframe dimension mismatch. Cannot interpolate. ---")
            print(f"M0: {M0.shape}, M1: {M1.shape}, Current: {current_brain_shape}"); return

        stress_before_dream = self.cns.stress_level

        print("...Interpolating dream state using Hilbert (phase-aware) interpolation...")
        # Use multiple interpolation points for richer consolidation
        interpolation_points = [0.25, 0.5, 0.75]
        M_dreams = []
        for t in interpolation_points:
            M_dream = self.interpolator.interpolate(M0, M1, t=t, mode='hilbert')
            M_dreams.append(M_dream)

        # Average the interpolated states for smoother consolidation
        M_consolidated = np.mean(M_dreams, axis=0)
        self._load_brain_state_vector(M_consolidated)

        print("...Consolidating dream state (multiple interpolation points)...")
        # Process multiple dream queries for better consolidation
        dream_queries = ["... (dreaming) ...", "... (memory consolidation) ...", "... (neural replay) ..."]
        for dream_query in dream_queries:
            await self.process_query(dream_query, update_stress=False)

        stress_after_consolidation = self.cns.stress_level
        stress_reduction_factor = 0.25
        stress_reduction = max(0.0, stress_before_dream * stress_reduction_factor)
        self.cns.stress_level = max(0.0, stress_after_consolidation - stress_reduction)

        self.cns.apply_consolidation(factor=0.7)

        if self.cns.stress_level < 1.0:
            self.cns.set_consciousness(ConsciousnessLevel.ALERT)

        print(f"--- 💤 SLEEP CYCLE COMPLETE (Stress reduced by {stress_reduction:.4f}, from {stress_before_dream:.4f} to {self.cns.stress_level:.4f}, consolidation factor: {self.cns.consolidation_factor:.2f}) ---")

# --- 15. PyTorch Models (for the "School") ---
class LinearTorchModel(nn.Module):
    """"""
    def __init__(self, input_dim: int, num_classes: int):
        super().__init__(); self.fc = nn.Linear(input_dim, num_classes)
    def forward(self, x): return self.fc(x)



print("✅ Cell 55: All Aura 7.0 classes defined.")
class MultiTaskHead(nn.Module):
    """"""
    def __init__(self, input_dim: int, num_emotions: int, num_intents: int, num_tones: int):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.15)
        )
        self.emotion_head = nn.Linear(512, num_emotions)
        self.intent_head = nn.Linear(512, num_intents)
        self.tone_head = nn.Linear(512, num_tones)
    def forward(self, x):
        h = self.shared(x)
        return self.emotion_head(h), self.intent_head(h), self.tone_head(h)

def multitask_loss(emotion_out, emotion_y, intent_out, intent_y, tone_out, tone_y,
                  criterion_emotion, criterion_intent, criterion_tone):
    """ - Modified for weighted loss"""
    loss = (1.0 * criterion_emotion(emotion_out, emotion_y) +
            0.5 * criterion_intent(intent_out, intent_y) +
            0.5 * criterion_tone(tone_out, tone_y))
    return loss

print("✅ Cell 55: All Aura 7.0 classes defined.")

# --- This is the test cell for Aura 7.0 ---
# It must be run in a cell *after* Cell 55

# --- 1. Define Global Save Paths ---
SAVE_DIR = "/content/drive/MyDrive/aura_education_v7_final"
os.makedirs(SAVE_DIR, exist_ok=True)
THALAMUS_PATH = os.path.join(SAVE_DIR, "thalamic_router_multitask.pt")
LABEL_MAPS_PATH = os.path.join(SAVE_DIR, "aura_7_label_maps.json")

# --- 2. The Main Async Test Loop ---
async def main_test_loop():

    # --- 3. INITIALIZE AND EDUCATE ---
    print("--- 🧠 Initializing Aura 7.0 (Loading LLMs with Unsloth...) ---")
    aura = IntegratedBioNeuralNetwork(
        n_experts=15,
        hebbian_components=64, # Initial size
        llm_model_name="meta-llama/Llama-3.2-3B-Instruct"
    )
    print("--- Aura LLM Components Loaded ---")

    # Run the "School"
    await aura.educate_brain() # This will train on go_emotions

    # --- 4. TEST 1: (PRE-TRAINED, "happy") ---
    print("\n\n--- 🗣️ TEST 1: Educated Brain, 'Happy' Query ---")
    response_1 = await aura.process_query("I am so happy and full of joy!")
    print(f"\nFINAL AURA RESPONSE (Happy): {response_1}")
    await aura.save_keyframe("happy") # Save this brain state
    day_1_stress = aura.last_run_final_stress

    # --- 5. TEST 2: (PRE-TRAINED, "scared") ---
    print("\n\n--- 🗣️ TEST 2: Educated Brain, 'Scared' Query ---")
    response_2 = await aura.process_query("I am feeling very scared")
    print(f"\nFINAL AURA RESPONSE (Scared): {response_2}")
    await aura.save_keyframe("scared") # Save this brain state
    day_2_stress = aura.last_run_final_stress

    # --- 6. "DREAM" CYCLE ---
    await aura.run_sleep_cycle(keyframe1_name="happy", keyframe2_name="scared")

    # --- 7. TEST 3: (POST-DREAM, "scared") ---
    print("\n\n--- 🗣️ TEST 3: Post-Dream Brain, 'Scared' Query (Day 3) ---")
    response_3 = await aura.process_query("I am feeling very scared")
    print(f"\nFINAL AURA RESPONSE (Scared, Day 3): {response_3}")
    day_3_stress = aura.last_run_final_stress

    # --- 8. VERIFICATION ---
    print("\n\n--- 🔬 TEMPORAL LEARNING VERIFICATION ---")
    print(f"Day 2 Final Stress (on 'scared'): {day_2_stress:.4f}")
    print(f"Day 3 Final Stress (on 'scared'): {day_3_stress:.4f}")

    if day_3_stress < (day_2_stress * 0.95): # Look for any reduction
        print("✅ SUCCESS: The 'dream' cycle helped process the fear! Stress is lower.")
    else:
        print("❌ NEUTRAL/FAILURE: The dream cycle did not reduce stress.")

# --- Benchmark Test Suite for Standard Datasets ---
# Optimized for 22GB VRAM (L4 GPU):
# - Increased batch sizes (64->256 for training, 64->256 for SBERT)
# - Larger datasets (15K->30K for GoEmotions, 5K->20K for MNIST, 3K->10K for CIFAR-10)
# - More experts (10->15) for better capacity
# - More warm-up samples (500->2000) for better MoE learning
# - Periodic GPU cache clearing to prevent OOM

class ImageFeatureGenerator:
    """Feature generator for image datasets like MNIST"""
    def __init__(self, image_size: int = 28):
        self.image_size = image_size
        self.feature_dim = image_size * image_size
        print(f"CREATED: ImageFeatureGenerator for {image_size}x{image_size} images")

    def extract_features(self, image: np.ndarray) -> np.ndarray:
        """Extract features from image (flatten and normalize)"""
        if len(image.shape) > 1:
            image = image.flatten()
        image = image.astype(np.float32) / 255.0
        return image

class SimplifiedAuraForBenchmark:
    """Simplified Aura system for benchmark testing on standard datasets"""
    def __init__(self, n_experts: int = 10, hebbian_components: int = 64,
                 feature_dim: int = 784, num_classes: int = 10):
        self.feature_dim = feature_dim
        self.num_classes = num_classes
        self.abstract_feature_dim = hebbian_components
        self.n_experts = n_experts

        self.whitener = OptimizedWhitener(dim=feature_dim)
        self.hippocampus = OjaLayer(
            dim=feature_dim,
            n_components=hebbian_components,
            mode='nonlinear', lr=5e-4, max_components=128, grow_threshold=0.3
        )

        self.cortex = None

    def _create_experts(self, hebbian_k: int):
        """Create MoE experts with the correct dimension"""
        experts: Dict[str, NLMSExpertAdapter] = {}
        attention_config = {
            'decay': 0.7,
            'theta': 1.0,
            'k_winners': 3,
            'gain_up': 1.5,
            'gain_down': 0.7,
            'multi_channel': {
                'k_winners': 5,
                'w_amp': 1.0,
                'w_pitch': 1.4,
                'w_bound': 0.8,
                'gain_up': 2.0,
                'gain_down': 0.5,
                'use_stdp': True,
                'smoothing': 1,
            }
        }
        for i in range(self.n_experts):
            name = f"expert__{i}"
            # Initialize with bias toward middle of class range (0-9 for MNIST/CIFAR)
            target_class_idx = (i + 0.5) * (10.0 / self.n_experts)
            head = ExpertNLMSHead(
                n_features=hebbian_k,
                vocab_size=1000,
                attention_config=attention_config,
                mu=0.5,  # Even higher learning rate
                mu_decay=0.99995,  # Very slow decay
                mu_min=0.1,  # Higher minimum
                initial_bias=target_class_idx  # Initialize bias to target range
            )
            # Initialize weights with larger random values
            head.w = np.random.normal(0, 1.0, hebbian_k)
            # Ensure bias is actually set
            head.bias = float(target_class_idx)
            experts[name] = NLMSExpertAdapter(neuron=head)

        self.cortex = LiquidMoERouter(
            experts=experts, in_dim=hebbian_k,
            hidden_dim=128, top_k=3
        )
        self.abstract_feature_dim = hebbian_k

    async def train_and_evaluate(self, X_train: np.ndarray, y_train: np.ndarray,
                                 X_test: np.ndarray, y_test: np.ndarray,
                                 epochs: int = 5) -> Dict[str, float]:
        """Train on dataset and return evaluation metrics"""
        print(f"Training on {len(X_train)} samples, testing on {len(X_test)} samples...")

        # Train Hebbian layer
        print("Training Hebbian Cortex (OjaLayer)...")
        X_train_whitened = np.array([self.whitener.transform(x) for x in X_train])
        for x_w in X_train_whitened:
            self.hippocampus.step(x_w)

        # Get final K after potential growth
        final_k = self.hippocampus.K
        print(f"Hebbian Cortex final K: {final_k}")

        # Create experts with correct dimension
        self._create_experts(final_k)

        # Warm-up MoE experts
        print("Warming up MoE experts...")
        warmup_samples = min(2000, len(X_train))
        val_indices = np.random.choice(len(X_test), min(500, len(X_test)), replace=False)
        errors = []
        for sweep in range(epochs):
            print(f"  Warm-up sweep {sweep+1}/{epochs}")
            epoch_errors = []
            indices = np.random.permutation(len(X_train))
            for idx in indices[:warmup_samples]:
                x = X_train[idx]
                y_true = float(y_train[idx])

                x_w = self.whitener.transform(x)
                oja_out = self.hippocampus.step(x_w)
                y_abstract = oja_out.y

                token_ids = list(range(min(10, len(x))))
                if self.cortex is None:
                    raise RuntimeError("Cortex not initialized. Call _create_experts() first.")
                out = await self.cortex.learn(x=y_abstract, token_ids=token_ids, y_true=y_true)

                pred = out['y_hat']
                _, discrete_pred = quantize_moe_prediction(pred, self.num_classes)
                error = abs(y_true - discrete_pred)
                epoch_errors.append(error)

            avg_error = np.mean(epoch_errors)
            errors.append(avg_error)
            print(f"    Sweep {sweep+1}: Avg Error = {avg_error:.4f}")

            # Validation on held-out test samples
            val_errors = []
            val_predictions = []
            val_targets = []
            for vidx in val_indices:
                x_val = X_test[int(vidx)]
                y_val = float(y_test[int(vidx)])
                x_vw = self.whitener.transform(x_val)
                oja_out = self.hippocampus.step(x_vw)
                y_abs = oja_out.y
                token_ids = list(range(min(10, len(x_val))))
                out = await self.cortex.route(x=y_abs, attn_gain=1.0)
                pred_val = out['y_hat']
                _, discrete_val = quantize_moe_prediction(pred_val, self.num_classes)
                val_errors.append(abs(y_val - discrete_val))
                val_predictions.append(discrete_val)
                val_targets.append(y_val)
            if val_errors:
                val_mae = np.mean(val_errors)
                val_rmse = np.sqrt(np.mean([e*e for e in val_errors]))
                val_corr = np.corrcoef(val_predictions, val_targets)[0, 1] if len(val_predictions) > 1 else 0.0
                print(f"    Validation -> MAE={val_mae:.4f}, RMSE={val_rmse:.4f}, Corr={val_corr:.3f}")

        # Evaluate on test set
        print("Evaluating on test set...")
        test_predictions = []
        test_targets = []
        test_errors = []

        for i in range(min(2000, len(X_test))):
            x = X_test[i]
            y_true = float(y_test[i])

            x_w = self.whitener.transform(x)
            oja_out = self.hippocampus.step(x_w)
            y_abstract = oja_out.y

            token_ids = list(range(min(10, len(x))))
            if self.cortex is None:
                raise RuntimeError("Cortex not initialized. Call _create_experts() first.")
            out = await self.cortex.route(x=y_abstract, attn_gain=1.0)

            pred = out['y_hat']
            pred_idx, discrete_pred = quantize_moe_prediction(pred, self.num_classes)
            error = abs(y_true - discrete_pred)

            test_predictions.append(discrete_pred)
            test_targets.append(y_true)
            test_errors.append(error)

        mae = np.mean(test_errors)
        rmse = np.sqrt(np.mean([e*e for e in test_errors]))

        # Calculate accuracy (rounded and clipped predictions)
        rounded_preds = np.round(test_predictions)
        # Clip to valid class range [0, num_classes-1]
        rounded_preds = np.clip(rounded_preds, 0, self.num_classes - 1).astype(int)
        test_targets_int = np.array(test_targets, dtype=int)
        accuracy = np.mean(rounded_preds == test_targets_int)

        correlation = np.corrcoef(test_predictions, test_targets)[0, 1] if len(test_predictions) > 1 else 0.0

        return {
            'mae': mae,
            'rmse': rmse,
            'accuracy': accuracy,
            'correlation': correlation,
            'final_train_error': errors[-1] if errors else float('inf')
        }

class NewsPatternAnalyzer:
    """Tracks topic/entity trends over rolling windows for news ingestion."""
    def __init__(self, max_history: int = 200):
        self.topic_history = defaultdict(lambda: deque(maxlen=max_history))

    def update_from_article(self, article: Dict[str, Any], prediction: int, discrete_value: float):
        topic = article.get('topic') or article.get('category') or 'general'
        entry = {
            'timestamp': article.get('timestamp', time.time()),
            'prediction': int(prediction),
            'value': float(discrete_value),
            'headline': article.get('headline', '')[:120]
        }
        self.topic_history[topic].append(entry)

    def summarize_topic(self, topic: str = 'general') -> Dict[str, Any]:
        history = list(self.topic_history.get(topic, []))
        if not history:
            return {}
        values = [item['value'] for item in history]
        predictions = [item['prediction'] for item in history]
        return {
            'topic': topic,
            'sample_count': len(history),
            'avg_value': float(np.mean(values)),
            'latest_prediction': predictions[-1],
            'latest_headline': history[-1].get('headline', '')
        }

class HistoricalTimelineAnalyzer:
    """Maintains per-entity timelines for pattern matching."""
    def __init__(self):
        self.timelines = defaultdict(list)

    def register_entity(self, entity: str, events: List[Dict[str, Any]]):
        normalized = []
        for event in events:
            normalized.append({
                'timestamp': event.get('timestamp', time.time()),
                'summary': event.get('summary', ''),
                'sentiment': event.get('sentiment', 0.0)
            })
        self.timelines[entity].extend(normalized)

    def compare_pattern(self, query: str, window: int = 5) -> Dict[str, Any]:
        tokens = query.lower().split()
        entity = tokens[0] if tokens else 'unknown'
        events = self.timelines.get(entity)
        if not events:
            return {}
        recent = events[-window:]
        avg_sentiment = float(np.mean([e.get('sentiment', 0.0) for e in recent]))
        return {
            'entity': entity,
            'samples_compared': len(recent),
            'avg_sentiment': avg_sentiment,
            'recent_events': recent
        }

class NewsIngestionService:
    """Fetches/processes news batches and updates analyzers."""
    def __init__(self, pattern_analyzer: Optional[NewsPatternAnalyzer] = None):
        self.pattern_analyzer = pattern_analyzer or NewsPatternAnalyzer()

    async def fetch_latest_batch(self) -> List[Dict[str, Any]]:
        """Placeholder for real API calls."""
        print("⚠️ fetch_latest_batch is not implemented; returning empty batch.")
        return []

    async def process_articles(self, aura: IntegratedBioNeuralNetwork, articles: List[Dict[str, Any]]):
        processed = []
        for article in articles:
            text = article.get('text') or article.get('headline') or ''
            if not text:
                continue
            target_signals = aura.router.get_target_signals(text, verbose=False)
            target_idx = float(target_signals['emotion'])
            thalamus_label = aura.router.emotion_labels_inv.get(int(target_idx), 'neutral')

            x_raw, token_ids, attention_bundle = aura.feature_gen.generate_for_query(text, thalamus_prediction=thalamus_label)
            xw = aura.feature_gen.whitener.transform(x_raw)
            oja_out = aura.hippocampus.step(xw)
            y_abstract = oja_out.y

            out = await aura.cortex.learn(
                x=y_abstract,
                token_ids=token_ids,
                y_true=target_idx,
                attention_bundle=attention_bundle
            )
            pred_idx, discrete_pred = quantize_moe_prediction(out['y_hat'], NUM_GOEMOTIONS)
            article_result = {
                'article': article,
                'prediction_idx': pred_idx,
                'prediction_value': discrete_pred
            }
            processed.append(article_result)
            if self.pattern_analyzer:
                self.pattern_analyzer.update_from_article(article, pred_idx, discrete_pred)
        return processed

async def test_mnist():
    """Test Aura on MNIST dataset"""
    print("\n" + "="*70)
    print("--- 🧪 BENCHMARK TEST: MNIST Dataset ---")
    print("="*70)

    try:
        from torchvision import datasets, transforms
        from torch.utils.data import DataLoader

        transform = transforms.Compose([transforms.ToTensor()])
        mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
        mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

        # Convert to numpy (using more data with 22GB VRAM)
        X_train = []
        y_train = []
        for i in range(min(20000, len(mnist_train))):
            img, label = mnist_train[i]
            X_train.append(img.numpy().flatten())
            y_train.append(label)

        X_test = []
        y_test = []
        for i in range(min(5000, len(mnist_test))):
            img, label = mnist_test[i]
            X_test.append(img.numpy().flatten())
            y_test.append(label)

        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        print(f"Loaded MNIST: {len(X_train)} train, {len(X_test)} test samples")

        aura_bench = SimplifiedAuraForBenchmark(
            n_experts=15,
            hebbian_components=64,
            feature_dim=784,
            num_classes=10
        )

        results = await aura_bench.train_and_evaluate(
            X_train, y_train, X_test, y_test, epochs=5
        )

        print("\n--- 📊 MNIST Results ---")
        print(f"Test Accuracy: {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
        print(f"Test MAE: {results['mae']:.4f}")
        print(f"Test RMSE: {results['rmse']:.4f}")
        print(f"Correlation: {results['correlation']:.4f}")
        print(f"Final Train Error: {results['final_train_error']:.4f}")

        return results

    except ImportError:
        print("❌ torchvision not available. Install with: pip install torchvision")
        return None
    except Exception as e:
        print(f"❌ Error testing MNIST: {e}")
        return None

async def test_cifar10():
    """Test Aura on CIFAR-10 dataset"""
    print("\n" + "="*70)
    print("--- 🧪 BENCHMARK TEST: CIFAR-10 Dataset ---")
    print("="*70)

    try:
        from torchvision import datasets, transforms
        from torch.utils.data import DataLoader

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Grayscale()
        ])
        cifar_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        cifar_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

        # Convert to numpy (using more data with 22GB VRAM)
        X_train = []
        y_train = []
        for i in range(min(10000, len(cifar_train))):
            img, label = cifar_train[i]
            X_train.append(img.numpy().flatten())
            y_train.append(label)

        X_test = []
        y_test = []
        for i in range(min(2000, len(cifar_test))):
            img, label = cifar_test[i]
            X_test.append(img.numpy().flatten())
            y_test.append(label)

        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        print(f"Loaded CIFAR-10: {len(X_train)} train, {len(X_test)} test samples")

        aura_bench = SimplifiedAuraForBenchmark(
            n_experts=15,
            hebbian_components=64,
            feature_dim=32*32,
            num_classes=10
        )

        results = await aura_bench.train_and_evaluate(
            X_train, y_train, X_test, y_test, epochs=5
        )

        print("\n--- 📊 CIFAR-10 Results ---")
        print(f"Test Accuracy: {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
        print(f"Test MAE: {results['mae']:.4f}")
        print(f"Test RMSE: {results['rmse']:.4f}")
        print(f"Correlation: {results['correlation']:.4f}")

        return results

    except ImportError:
        print("❌ torchvision not available. Install with: pip install torchvision")
        return None
    except Exception as e:
        print(f"❌ Error testing CIFAR-10: {e}")
        return None

async def run_all_benchmarks():
    """Run all benchmark tests"""
    print("\n" + "="*70)
    print("--- 🚀 RUNNING ALL BENCHMARK TESTS ---")
    print("="*70)

    results = {}

    # Test MNIST
    mnist_results = await test_mnist()
    if mnist_results:
        results['MNIST'] = mnist_results

    # Test CIFAR-10
    cifar_results = await test_cifar10()
    if cifar_results:
        results['CIFAR-10'] = cifar_results

    # Summary
    if results:
        print("\n" + "="*70)
        print("--- 📈 BENCHMARK SUMMARY ---")
        print("="*70)
        for dataset_name, res in results.items():
            print(f"\n{dataset_name}:")
            print(f"  Accuracy: {res['accuracy']*100:.2f}%")
            print(f"  MAE: {res['mae']:.4f}")
            print(f"  RMSE: {res['rmse']:.4f}")

    return results

# --- Helper Functions for Running Tests ---
async def run_mnist_test_only():
    """Run only MNIST benchmark test"""
    return await test_mnist()

async def run_cifar10_test_only():
    """Run only CIFAR-10 benchmark test"""
    return await test_cifar10()

# --- Run the async main loop ---
import nest_asyncio
nest_asyncio.apply()

# Set RUN_BENCHMARKS=True to run benchmark tests instead of main test loop
# Or use: RUN_BENCHMARKS="mnist" for MNIST only, "cifar10" for CIFAR-10 only
RUN_BENCHMARKS = False

if RUN_BENCHMARKS == True:
    print("="*70)
    print("Running All Benchmark Tests (MNIST, CIFAR-10)")
    print("="*70)
    asyncio.run(run_all_benchmarks())
elif RUN_BENCHMARKS == "mnist":
    print("="*70)
    print("Running MNIST Benchmark Test")
    print("="*70)
    asyncio.run(run_mnist_test_only())
elif RUN_BENCHMARKS == "cifar10":
    print("="*70)
    print("Running CIFAR-10 Benchmark Test")
    print("="*70)
    asyncio.run(run_cifar10_test_only())
else:
    asyncio.run(main_test_loop())

Hugging Face login successful!
Hugging Face login successful!
✅ Cell 55: All Aura 7.0 classes defined.
✅ Cell 55: All Aura 7.0 classes defined.
--- 🧠 Initializing Aura 7.0 (Loading LLMs with Unsloth...) ---
--- 🧠 Initializing Aura 7.0 (Hebbian-MoE-Temporal) ---
   -> GPU: NVIDIA L4, VRAM: 23.8GB
CREATED: FeatureGenerator (Aura 7.0 Ears), Features: 419
CREATED: ResponseGenerator (Mouth) using Unsloth on 'meta-llama/Llama-3.2-3B-Instruct'
   -> Using optimized Unsloth 4-bit model: unsloth/Llama-3.2-3B-Instruct-bnb-4bit
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
   -> Unsloth Llama 

# TEST v7.0 (AURA v1)
